In [2]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [3]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_luciano_ml"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-12-01T19:32:31.478108Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-12-01T19:32:31.479343Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-12-01T19:32:31.479733Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-12-01T19:32:31.480191Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-12-01T19:32:31.480590Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-23fdd520-5874-4005-9439-05a6f3fb5dd6;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.19.0.0 in central
	found com.oracle.databa

In [6]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = 179b86d5-08b0-444f-9677-656d3dc4e94a


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [4]:
# ============================================================================
# CONFIGURAÇÃO INICIAL - 
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    concat as spark_concat,
    lit as spark_lit,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    countDistinct as spark_countDistinct
)
from pyspark.sql.types import DoubleType, IntegerType

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# ✅ CORREÇÃO: Não usar abs() que conflita com PySpark
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}' if abs(x) > 0.01 else f'{x:.6f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Acesso ao Spark
spark = session.sparkSession

print("=" * 80)
print("🔍 SISTEMA")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

🔍 SISTEMA
Sessão Spark: tsevero_luciano_ml
Versão Spark: 3.5.4.3.5.7191000.0-30
Iniciado em: 2025-12-01 16:32:47


In [7]:
# ==============================================================================
# CÉLULA 1: QUERY DE FEATURES EXPANDIDA (CORRIGIDA)
# ==============================================================================
# Versão corrigida que:
# - Usa ods_pagamento_raw em vez de vw_ods_pagamento (view com erro)
# - Usa cad_vinculo_raw em vez de vw_cad_vinculo (view com erro)
# ==============================================================================

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🤖 MODELO ML - PREDIÇÃO DE CANCELAMENTOS (FEATURES EXPANDIDAS)")
print("="*80)
print(f"Início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ===================================================================
# PASSO 1: PREPARAR DADOS DE TREINO
# ===================================================================

print("\n" + "="*80)
print("📊 PASSO 1: PREPARANDO DADOS DE TREINO")
print("="*80)

query_treino = """
WITH empresas_canceladas AS (
    SELECT DISTINCT
        s.cnpj,
        s.flag_atualmente_cancelada,
        s.flag_empresa_reincidente,
        CASE 
            WHEN s.flag_atualmente_cancelada = 1 THEN 1
            ELSE 0
        END AS target_deve_cancelar
    FROM teste.luciano_scores s
),
empresas_nunca_canceladas AS (
    SELECT DISTINCT
        c.nu_cnpj AS cnpj,
        0 AS flag_atualmente_cancelada,
        0 AS flag_empresa_reincidente,
        0 AS target_deve_cancelar
    FROM usr_sat_ods.vw_ods_contrib c
    WHERE c.nm_sit_cadastral = 'ATIVO'
        AND c.nu_cnpj NOT IN (
            SELECT DISTINCT cnpj 
            FROM teste.luciano_scores
        )
    LIMIT 50000
),
base_treino AS (
    SELECT * FROM empresas_canceladas
    UNION ALL
    SELECT * FROM empresas_nunca_canceladas
)
SELECT * FROM base_treino
"""

print("🔍 Buscando empresas para treino...")
df_treino_base = spark.sql(query_treino).toPandas()

print(f"\n✅ Base de treino carregada:")
print(f"   Total: {len(df_treino_base):,} empresas")
print(f"\n📊 Distribuição do target:")
print(df_treino_base['target_deve_cancelar'].value_counts())

# ===================================================================
# PASSO 2: BUSCAR FEATURES
# ===================================================================

print("\n" + "="*80)
print("🔧 PASSO 2: ENRIQUECENDO COM FEATURES EXPANDIDAS")
print("="*80)

cnpjs_treino = df_treino_base['cnpj'].tolist()

def criar_filtro_cnpjs(cnpjs, batch_size=10000):
    batches = [cnpjs[i:i + batch_size] for i in range(0, len(cnpjs), batch_size)]
    return batches

print(f"📦 Dividindo {len(cnpjs_treino):,} CNPJs em batches...")
batches = criar_filtro_cnpjs(cnpjs_treino, batch_size=10000)

# ===================================================================
# QUERY DE FEATURES EXPANDIDA (CORRIGIDA)
# - Usa ods_pagamento_raw em vez de vw_ods_pagamento
# - Usa cad_vinculo_raw em vez de vw_cad_vinculo
# ===================================================================

query_features_expandida = """
WITH cnpjs_treino AS (
    SELECT cnpj FROM (
        VALUES {cnpjs_values}
    ) AS t(cnpj)
),

-- ============================================================================
-- 1. INDÍCIOS NEAF (expandido)
-- ============================================================================
indicios AS (
    SELECT
        ind.nu_cpf_cnpj AS cnpj,
        COUNT(*) AS qtd_indicios,
        SUM(COALESCE(ind.nu_score, 0)) AS soma_score_indicios,
        COUNT(DISTINCT ind.cd_indicio) AS qtd_tipos_indicios_distintos,
        MAX(CASE WHEN ind.cd_indicio = 11 THEN 1 ELSE 0 END) AS flag_omissao_dime,
        MAX(CASE WHEN ind.cd_indicio = 12 THEN 1 ELSE 0 END) AS flag_omissao_pgdas,
        MAX(CASE WHEN ind.cd_indicio IN (1,2,3,4,5) THEN 1 ELSE 0 END) AS flag_indicios_graves
    FROM neaf.empresa_indicio ind
    WHERE ind.nu_cpf_cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND ind.cd_atual = 1
    GROUP BY ind.nu_cpf_cnpj
),

-- ============================================================================
-- 2. PGDAS - SIMPLES NACIONAL (expandido)
-- ============================================================================
pgdas AS (
    SELECT
        p.nu_cnpj AS cnpj,
        SUM(p.vl_rec_bruta_estab) AS receita_12m,
        MAX(p.vl_rec_bruta_estab) AS receita_maxima,
        MIN(CASE WHEN p.vl_rec_bruta_estab > 0 THEN p.vl_rec_bruta_estab ELSE NULL END) AS receita_minima,
        AVG(p.vl_rec_bruta_estab) AS receita_media,
        STDDEV(p.vl_rec_bruta_estab) AS receita_desvio_padrao,
        COUNT(DISTINCT p.nu_per_ref) AS qtd_meses_declarados,
        SUM(CASE WHEN p.vl_rec_bruta_estab = 0 THEN 1 ELSE 0 END) AS meses_zerados_pgdas,
        MAX(p.nu_per_ref) AS ultimo_periodo_pgdas,
        MIN(p.nu_per_ref) AS primeiro_periodo_pgdas
    FROM usr_sat_ods.sna_pgdasd_estabelecimento_raw p
    WHERE p.nu_cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND p.nu_per_ref >= 202301
    GROUP BY p.nu_cnpj
),

-- ============================================================================
-- 3. DIME - REGIME NORMAL
-- ============================================================================
dime AS (
    SELECT
        d.nu_cnpj AS cnpj,
        SUM(d.vl_receita_bruta) AS receita_bruta_dime,
        SUM(d.vl_faturamento) AS faturamento_dime,
        SUM(d.vl_tot_cred) AS total_creditos_dime,
        SUM(d.vl_tot_deb) AS total_debitos_dime,
        SUM(d.vl_deb_recolher) AS debito_recolher_dime,
        SUM(d.vl_cred_mes_anterior) AS cred_mes_anterior_dime,
        SUM(d.vl_cred_mes_seguinte) AS cred_mes_seguinte_dime,
        COUNT(DISTINCT d.nu_per_ref) AS qtd_meses_dime,
        SUM(CASE WHEN d.vl_faturamento = 0 THEN 1 ELSE 0 END) AS meses_zerados_dime,
        SUM(CASE WHEN d.vl_tot_cred = 0 AND d.vl_tot_deb = 0 THEN 1 ELSE 0 END) AS meses_sem_movimento_dime,
        MAX(d.nu_per_ref) AS ultimo_periodo_dime
    FROM usr_sat_ods.ods_decl_dime_raw d
    WHERE d.nu_cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND d.nu_per_ref >= 202301
    GROUP BY d.nu_cnpj
),

-- ============================================================================
-- 4. RECOLHIMENTO/PAGAMENTOS ICMS (CORRIGIDO - usa tabela RAW)
-- ============================================================================
recolhimento AS (
    SELECT
        p.nu_cnpj AS cnpj,
        COUNT(p.nu_id_pagamento) AS qtd_pagamentos_icms,
        SUM(p.vl_total) AS valor_total_pagamentos,
        AVG(p.vl_total) AS valor_medio_pagamento,
        MAX(p.vl_total) AS maior_pagamento,
        COUNT(DISTINCT p.nu_per_pagamento) AS meses_com_pagamento,
        MAX(p.nu_per_pagamento) AS ultimo_periodo_pagamento
    FROM usr_sat_ods.ods_pagamento_raw p
    WHERE p.nu_cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND p.nu_per_pagamento >= 202301
        AND p.cd_receita IN (1449, 1465, 1473, 1554, 1570, 1589, 1597, 1600, 1643, 1651, 
                             1716, 1724, 1732, 1740, 1759, 1767, 1783, 1791, 2140, 2496, 
                             2526, 2534, 3000, 3050, 3662, 4383, 7110, 7137, 9687, 9784)
    GROUP BY p.nu_cnpj
),

-- ============================================================================
-- 5. MEIOS DE PAGAMENTO - DIMP
-- ============================================================================
dimp AS (
    SELECT
        ato_nu_cnpjmf AS cnpj,
        SUM(ato_vl_credito) AS total_credito_cartao,
        SUM(ato_vl_debito) AS total_debito_cartao,
        SUM(ato_vl_pix) AS total_pix,
        SUM(ato_vl_credito + ato_vl_debito + ato_vl_pix) AS total_meios_pagamento,
        COUNT(DISTINCT ato_dt_referencia) AS meses_com_meios_pagamento
    FROM usr_sat_admcc.acc_r66_totalestab
    WHERE ato_nu_cnpjmf IN (SELECT cnpj FROM cnpjs_treino)
        AND ato_dt_referencia >= 202301
        AND LENGTH(TRIM(ato_nu_cnpjmf)) = 14
    GROUP BY ato_nu_cnpjmf
),

-- ============================================================================
-- 6. NF-e EMITIDAS
-- ============================================================================
nfe_emit AS (
    SELECT
        procnfe.nfe.infnfe.emit.cnpj AS cnpj,
        COUNT(DISTINCT chave) AS qtd_nfes_emitidas,
        SUM(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_total_nfes,
        SUM(procnfe.nfe.infnfe.total.icmstot.vicms) AS icms_total_nfes,
        COUNT(DISTINCT procnfe.nfe.infnfe.dest.cnpj) AS qtd_destinatarios_distintos,
        AVG(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_medio_nfe
    FROM nfe.nfe
    WHERE procnfe.nfe.infnfe.emit.cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND (ano_emissao * 100 + mes_emissao) >= 202301
        AND procnfe.nfe.infnfe.ide.tpnf = 1
    GROUP BY procnfe.nfe.infnfe.emit.cnpj
),

-- ============================================================================
-- 7. NF-e RECEBIDAS
-- ============================================================================
nfe_receb AS (
    SELECT
        procnfe.nfe.infnfe.dest.cnpj AS cnpj,
        COUNT(DISTINCT chave) AS qtd_nfes_recebidas,
        SUM(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_total_nfes_recebidas,
        COUNT(DISTINCT procnfe.nfe.infnfe.emit.cnpj) AS qtd_fornecedores_distintos
    FROM nfe.nfe
    WHERE procnfe.nfe.infnfe.dest.cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND (ano_emissao * 100 + mes_emissao) >= 202301
    GROUP BY procnfe.nfe.infnfe.dest.cnpj
),

-- ============================================================================
-- 8. VÍNCULOS SOCIETÁRIOS (CORRIGIDO - usa tabela RAW)
-- ============================================================================
socios AS (
    SELECT
        v.nu_cnpj_princ AS cnpj,
        COUNT(DISTINCT COALESCE(v.nu_cnpj_secund, v.nu_cpf_secund)) AS qtd_socios,
        COUNT(DISTINCT CASE WHEN v.sn_relacao_ativa = 1 THEN COALESCE(v.nu_cnpj_secund, v.nu_cpf_secund) END) AS qtd_socios_ativos_vinculo,
        SUM(CASE WHEN v.nm_relacao = 'SOCIO' OR v.nm_relacao = 'SÓCIO' THEN 1 ELSE 0 END) AS qtd_tipo_socio,
        SUM(CASE WHEN v.nm_relacao = 'REPRESENTANTE' THEN 1 ELSE 0 END) AS qtd_representantes,
        MAX(v.pe_capital_empresa) AS maior_participacao,
        MIN(CASE WHEN v.pe_capital_empresa > 0 THEN v.pe_capital_empresa ELSE NULL END) AS menor_participacao,
        AVG(v.pe_capital_empresa) AS media_participacao,
        COUNT(DISTINCT CASE WHEN v.nu_cpf_secund IS NOT NULL THEN v.nu_cpf_secund END) AS qtd_socios_pf,
        COUNT(DISTINCT CASE WHEN v.nu_cnpj_secund IS NOT NULL THEN v.nu_cnpj_secund END) AS qtd_socios_pj
    FROM usr_sat_ods.cad_vinculo_raw v
    WHERE v.nu_cnpj_princ IN (SELECT cnpj FROM cnpjs_treino)
        AND v.nm_relacao NOT IN ('CONTABILISTA')
    GROUP BY v.nu_cnpj_princ
),

-- ============================================================================
-- 9. FUNCIONÁRIOS RAIS/CAGED
-- ============================================================================
funcionarios AS (
    SELECT
        r.cnpj_cei AS cnpj,
        COUNT(DISTINCT r.cpf) AS qtd_funcionarios,
        AVG(r.vl_remun_media_nom) AS salario_medio,
        SUM(r.vl_remun_media_nom) AS folha_total,
        MAX(r.vl_remun_media_nom) AS maior_salario,
        MIN(CASE WHEN r.vl_remun_media_nom > 0 THEN r.vl_remun_media_nom ELSE NULL END) AS menor_salario
    FROM rais_caged.vw_rais_vinculos r
    WHERE r.cnpj_cei IN (SELECT cnpj FROM cnpjs_treino)
        AND r.vinculo_ativo_31_12 = 1
        AND r.ano_arquivo >= 2023
    GROUP BY r.cnpj_cei
),

-- ============================================================================
-- 10. CONTAS BANCÁRIAS CCS
-- ============================================================================
contas_bancarias AS (
    SELECT
        c.nr_cnpj AS cnpj,
        COUNT(*) AS qtd_contas_bancarias,
        COUNT(DISTINCT c.nm_banco) AS qtd_bancos_distintos,
        COUNT(DISTINCT c.nr_cpf) AS qtd_responsaveis_contas,
        SUM(CASE WHEN c.dt_encerramento IS NULL THEN 1 ELSE 0 END) AS qtd_contas_ativas,
        SUM(CASE WHEN c.dt_encerramento IS NOT NULL THEN 1 ELSE 0 END) AS qtd_contas_encerradas
    FROM usr_sat_fsn.fsn_conta_bancaria c
    WHERE c.nr_cnpj IN (SELECT cnpj FROM cnpjs_treino)
        AND (c.valido IS NULL OR c.valido = 1)
    GROUP BY c.nr_cnpj
),

-- ============================================================================
-- 11. CADASTRO BÁSICO (expandido)
-- ============================================================================
cadastro AS (
    SELECT
        c.nu_cnpj AS cnpj,
        c.cd_cnae,
        c.nm_reg_apuracao,
        c.cd_uf,
        c.qt_socios_ativos,
        c.qt_vinculos_ativos,
        DATEDIFF(CURRENT_DATE(), c.dt_inicio_icms) AS dias_desde_inicio_icms,
        DATEDIFF(CURRENT_DATE(), c.dt_constituicao_empresa) AS dias_desde_constituicao,
        CASE WHEN c.nm_reg_apuracao = 'NORMAL' THEN 1 ELSE 0 END AS flag_regime_normal,
        CASE WHEN c.nm_reg_apuracao IN ('SIMPLES NACIONAL', 'SIMEI', 'SIMPLES SC') THEN 1 ELSE 0 END AS flag_simples,
        CASE WHEN c.cd_uf = 'SC' THEN 1 ELSE 0 END AS flag_uf_sc
    FROM usr_sat_ods.vw_ods_contrib c
    WHERE c.nu_cnpj IN (SELECT cnpj FROM cnpjs_treino)
)

-- ============================================================================
-- CONSOLIDAÇÃO FINAL
-- ============================================================================
SELECT
    ct.cnpj,
    
    -- INDÍCIOS (6 features)
    COALESCE(ind.qtd_indicios, 0) AS qtd_indicios,
    COALESCE(ind.soma_score_indicios, 0) AS soma_score_indicios,
    COALESCE(ind.qtd_tipos_indicios_distintos, 0) AS qtd_tipos_indicios_distintos,
    COALESCE(ind.flag_omissao_dime, 0) AS flag_omissao_dime,
    COALESCE(ind.flag_omissao_pgdas, 0) AS flag_omissao_pgdas,
    COALESCE(ind.flag_indicios_graves, 0) AS flag_indicios_graves,
    
    -- PGDAS (10 features)
    COALESCE(pg.receita_12m, 0) AS receita_12m,
    COALESCE(pg.receita_maxima, 0) AS receita_maxima,
    COALESCE(pg.receita_minima, 0) AS receita_minima,
    COALESCE(pg.receita_media, 0) AS receita_media,
    COALESCE(pg.receita_desvio_padrao, 0) AS receita_desvio_padrao,
    COALESCE(pg.qtd_meses_declarados, 0) AS qtd_meses_declarados,
    COALESCE(pg.meses_zerados_pgdas, 0) AS meses_zerados_pgdas,
    COALESCE(pg.ultimo_periodo_pgdas, 0) AS ultimo_periodo_pgdas,
    COALESCE(pg.primeiro_periodo_pgdas, 0) AS primeiro_periodo_pgdas,
    
    -- DIME (11 features)
    COALESCE(dm.receita_bruta_dime, 0) AS receita_bruta_dime,
    COALESCE(dm.faturamento_dime, 0) AS faturamento_dime,
    COALESCE(dm.total_creditos_dime, 0) AS total_creditos_dime,
    COALESCE(dm.total_debitos_dime, 0) AS total_debitos_dime,
    COALESCE(dm.debito_recolher_dime, 0) AS debito_recolher_dime,
    COALESCE(dm.cred_mes_anterior_dime, 0) AS cred_mes_anterior_dime,
    COALESCE(dm.cred_mes_seguinte_dime, 0) AS cred_mes_seguinte_dime,
    COALESCE(dm.qtd_meses_dime, 0) AS qtd_meses_dime,
    COALESCE(dm.meses_zerados_dime, 0) AS meses_zerados_dime,
    COALESCE(dm.meses_sem_movimento_dime, 0) AS meses_sem_movimento_dime,
    COALESCE(dm.ultimo_periodo_dime, 0) AS ultimo_periodo_dime,
    
    -- RECOLHIMENTO (6 features)
    COALESCE(rec.qtd_pagamentos_icms, 0) AS qtd_pagamentos_icms,
    COALESCE(rec.valor_total_pagamentos, 0) AS valor_total_pagamentos,
    COALESCE(rec.valor_medio_pagamento, 0) AS valor_medio_pagamento,
    COALESCE(rec.maior_pagamento, 0) AS maior_pagamento,
    COALESCE(rec.meses_com_pagamento, 0) AS meses_com_pagamento,
    COALESCE(rec.ultimo_periodo_pagamento, 0) AS ultimo_periodo_pagamento,
    
    -- DIMP - MEIOS DE PAGAMENTO (5 features)
    COALESCE(dp.total_credito_cartao, 0) AS total_credito_cartao,
    COALESCE(dp.total_debito_cartao, 0) AS total_debito_cartao,
    COALESCE(dp.total_pix, 0) AS total_pix,
    COALESCE(dp.total_meios_pagamento, 0) AS total_meios_pagamento,
    COALESCE(dp.meses_com_meios_pagamento, 0) AS meses_com_meios_pagamento,
    
    -- NF-e EMITIDAS (5 features)
    COALESCE(ne.qtd_nfes_emitidas, 0) AS qtd_nfes_emitidas,
    COALESCE(ne.valor_total_nfes, 0) AS valor_total_nfes,
    COALESCE(ne.icms_total_nfes, 0) AS icms_total_nfes,
    COALESCE(ne.qtd_destinatarios_distintos, 0) AS qtd_destinatarios_distintos,
    COALESCE(ne.valor_medio_nfe, 0) AS valor_medio_nfe,
    
    -- NF-e RECEBIDAS (3 features)
    COALESCE(nr.qtd_nfes_recebidas, 0) AS qtd_nfes_recebidas,
    COALESCE(nr.valor_total_nfes_recebidas, 0) AS valor_total_nfes_recebidas,
    COALESCE(nr.qtd_fornecedores_distintos, 0) AS qtd_fornecedores_distintos,
    
    -- SÓCIOS (10 features)
    COALESCE(soc.qtd_socios, 0) AS qtd_socios,
    COALESCE(soc.qtd_socios_ativos_vinculo, 0) AS qtd_socios_ativos_vinculo,
    COALESCE(soc.qtd_tipo_socio, 0) AS qtd_tipo_socio,
    COALESCE(soc.qtd_representantes, 0) AS qtd_representantes,
    COALESCE(soc.maior_participacao, 0) AS maior_participacao,
    COALESCE(soc.menor_participacao, 0) AS menor_participacao,
    COALESCE(soc.media_participacao, 0) AS media_participacao,
    COALESCE(soc.qtd_socios_pf, 0) AS qtd_socios_pf,
    COALESCE(soc.qtd_socios_pj, 0) AS qtd_socios_pj,
    
    -- FUNCIONÁRIOS (5 features)
    COALESCE(func.qtd_funcionarios, 0) AS qtd_funcionarios,
    COALESCE(func.salario_medio, 0) AS salario_medio,
    COALESCE(func.folha_total, 0) AS folha_total,
    COALESCE(func.maior_salario, 0) AS maior_salario,
    COALESCE(func.menor_salario, 0) AS menor_salario,
    
    -- CONTAS BANCÁRIAS (5 features)
    COALESCE(cb.qtd_contas_bancarias, 0) AS qtd_contas_bancarias,
    COALESCE(cb.qtd_bancos_distintos, 0) AS qtd_bancos_distintos,
    COALESCE(cb.qtd_responsaveis_contas, 0) AS qtd_responsaveis_contas,
    COALESCE(cb.qtd_contas_ativas, 0) AS qtd_contas_ativas,
    COALESCE(cb.qtd_contas_encerradas, 0) AS qtd_contas_encerradas,
    
    -- CADASTRO (8 features)
    cad.cd_cnae,
    cad.nm_reg_apuracao,
    cad.cd_uf,
    COALESCE(cad.qt_socios_ativos, 0) AS qt_socios_ativos,
    COALESCE(cad.qt_vinculos_ativos, 0) AS qt_vinculos_ativos,
    COALESCE(cad.dias_desde_inicio_icms, 0) AS dias_desde_inicio_icms,
    COALESCE(cad.dias_desde_constituicao, 0) AS dias_desde_constituicao,
    COALESCE(cad.flag_regime_normal, 0) AS flag_regime_normal,
    COALESCE(cad.flag_simples, 0) AS flag_simples,
    COALESCE(cad.flag_uf_sc, 0) AS flag_uf_sc

FROM cnpjs_treino ct
LEFT JOIN indicios ind ON ct.cnpj = ind.cnpj
LEFT JOIN pgdas pg ON ct.cnpj = pg.cnpj
LEFT JOIN dime dm ON ct.cnpj = dm.cnpj
LEFT JOIN recolhimento rec ON ct.cnpj = rec.cnpj
LEFT JOIN dimp dp ON ct.cnpj = dp.cnpj
LEFT JOIN nfe_emit ne ON ct.cnpj = ne.cnpj
LEFT JOIN nfe_receb nr ON ct.cnpj = nr.cnpj
LEFT JOIN socios soc ON ct.cnpj = soc.cnpj
LEFT JOIN funcionarios func ON ct.cnpj = func.cnpj
LEFT JOIN contas_bancarias cb ON ct.cnpj = cb.cnpj
LEFT JOIN cadastro cad ON ct.cnpj = cad.cnpj
"""

print("🔍 Buscando features expandidas...")
print("   Fontes: Indícios, PGDAS, DIME, Pagamentos, DIMP, NF-e, Sócios, Funcionários, Contas, Cadastro")

df_features_list = []
batches_sucesso = 0
batches_erro = 0

for i, batch in enumerate(batches, 1):
    print(f"   📦 Batch {i}/{len(batches)} ({len(batch):,} CNPJs)...", end=" ")
    cnpjs_values = ",".join([f"('{cnpj}')" for cnpj in batch])
    query = query_features_expandida.replace("{cnpjs_values}", cnpjs_values)
    
    try:
        df_batch = spark.sql(query).toPandas()
        df_features_list.append(df_batch)
        batches_sucesso += 1
        print("✅")
    except Exception as e:
        batches_erro += 1
        print(f"❌ Erro: {str(e)[:50]}...")
        continue

print(f"\n📊 Resumo:")
print(f"   ✅ Sucesso: {batches_sucesso}/{len(batches)}")
print(f"   ❌ Erro: {batches_erro}/{len(batches)}")

if df_features_list:
    df_features = pd.concat(df_features_list, ignore_index=True)
    print(f"\n✅ Features consolidadas:")
    print(f"   Registros: {len(df_features):,}")
    print(f"   Colunas: {len(df_features.columns)}")
    print(f"\n📋 Colunas disponíveis:")
    for col in df_features.columns:
        print(f"      - {col}")
    
    # Merge com base de treino
    df_treino = df_treino_base.merge(df_features, on='cnpj', how='left')
    
    print(f"\n✅ Base de treino enriquecida:")
    print(f"   Shape: {df_treino.shape}")
    
    # Salvar para análise
    print(f"\n💾 Salvando base de treino...")
    df_treino.to_csv('df_treino_features_expandidas.csv', index=False)
    print(f"   ✅ Salvo em: /home/claude/df_treino_features_expandidas.csv")
    
else:
    print("\n❌ ERRO: Nenhum batch processado com sucesso!")
    print("   Todas as queries falharam!")

print("\n" + "="*80)
print("✅ PASSO 1 E 2 CONCLUÍDOS!")
print("="*80)

🤖 MODELO ML - PREDIÇÃO DE CANCELAMENTOS (FEATURES EXPANDIDAS)
Início: 2025-11-27 14:20:50

📊 PASSO 1: PREPARANDO DADOS DE TREINO
🔍 Buscando empresas para treino...



✅ Base de treino carregada:
   Total: 293,053 empresas

📊 Distribuição do target:
target_deve_cancelar
1    232431
0     60622
Name: count, dtype: int64

🔧 PASSO 2: ENRIQUECENDO COM FEATURES EXPANDIDAS
📦 Dividindo 293,053 CNPJs em batches...
🔍 Buscando features expandidas...
   Fontes: Indícios, PGDAS, DIME, Pagamentos, DIMP, NF-e, Sócios, Funcionários, Contas, Cadastro
   📦 Batch 1/30 (10,000 CNPJs)... 

                                                                                1]]2]

✅
   📦 Batch 2/30 (10,000 CNPJs)... 

                                                                                / 1]28]

...

✅
   📦 Batch 29/30 (10,000 CNPJs)... 

                                                                                9]9]/ 1]]8]

✅
   📦 Batch 30/30 (3,053 CNPJs)... 

                                                                                ]8]]/ 1]]8]

✅

📊 Resumo:
   ✅ Sucesso: 30/30
   ❌ Erro: 0/30

✅ Features consolidadas:
   Registros: 293,053
   Colunas: 75

📋 Colunas disponíveis:
      - cnpj
      - qtd_indicios
      - soma_score_indicios
      - qtd_tipos_indicios_distintos
      - flag_omissao_dime
      - flag_omissao_pgdas
      - flag_indicios_graves
      - receita_12m
      - receita_maxima
      - receita_minima
      - receita_media
      - receita_desvio_padrao
      - qtd_meses_declarados
      - meses_zerados_pgdas
      - ultimo_periodo_pgdas
      - primeiro_periodo_pgdas
      - receita_bruta_dime
      - faturamento_dime
      - total_creditos_dime
      - total_debitos_dime
      - debito_recolher_dime
      - cred_mes_anterior_dime
      - cred_mes_seguinte_dime
      - qtd_meses_dime
      - meses_zerados_dime
      - meses_sem_movimento_dime
      - ultimo_periodo_dime
      - qtd_pagamentos_icms
      - valor_total_pagamentos
      - valor_medio_pagamento
      - maior_pagamento
      - meses_com_pagament

In [8]:
# ==============================================================================
# CÉLULA 2: FEATURE ENGINEERING (COMPATÍVEL COM QUERY CORRIGIDA)
# ==============================================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from decimal import Decimal

print("="*80)
print("🔧 PASSO 2B: CRIANDO FEATURES ENGENHEIRADAS")
print("="*80)

# ===================================================================
# CONVERSÃO DE TIPOS DECIMAL -> FLOAT
# ===================================================================

print("\n🔧 Convertendo tipos Decimal para float...")

# Identificar colunas numéricas que podem ser Decimal
for col in df_treino.columns:
    # Verificar se algum valor é Decimal
    if df_treino[col].apply(lambda x: isinstance(x, Decimal)).any():
        df_treino[col] = df_treino[col].apply(lambda x: float(x) if isinstance(x, Decimal) else x)
        print(f"   Convertido: {col}")

# Converter todas as colunas object que são numéricas
numeric_cols_to_convert = [
    'receita_12m', 'receita_maxima', 'receita_minima', 'receita_media', 'receita_desvio_padrao',
    'receita_bruta_dime', 'faturamento_dime', 'total_creditos_dime', 'total_debitos_dime',
    'debito_recolher_dime', 'cred_mes_anterior_dime', 'cred_mes_seguinte_dime',
    'valor_total_pagamentos', 'valor_medio_pagamento', 'maior_pagamento',
    'total_credito_cartao', 'total_debito_cartao', 'total_pix', 'total_meios_pagamento',
    'valor_total_nfes', 'icms_total_nfes', 'valor_medio_nfe',
    'valor_total_nfes_recebidas',
    'maior_participacao', 'menor_participacao', 'media_participacao',
    'salario_medio', 'folha_total', 'maior_salario', 'menor_salario',
    'soma_score_indicios'
]

for col in numeric_cols_to_convert:
    if col in df_treino.columns:
        df_treino[col] = pd.to_numeric(df_treino[col], errors='coerce').fillna(0).astype(float)

print("   ✅ Conversão de tipos concluída")

# ===================================================================
# FUNÇÃO DE FEATURE ENGINEERING
# ===================================================================

def criar_features_engenheiradas(df):
    """
    Cria features derivadas a partir das features básicas.
    Compatível com a query CELULA_1_CORRIGIDA.
    """
    df = df.copy()
    
    # =========================================================================
    # GRUPO 1: FEATURES DE RECEITA
    # =========================================================================
    
    # Receita total consolidada (PGDAS + DIME)
    df['receita_total_consolidada'] = df['receita_12m'] + df['receita_bruta_dime']
    
    # Flags de receita
    df['tem_receita_pgdas'] = (df['receita_12m'] > 0).astype(int)
    df['tem_receita_dime'] = (df['receita_bruta_dime'] > 0).astype(int)
    df['tem_receita'] = ((df['receita_12m'] > 0) | (df['receita_bruta_dime'] > 0)).astype(int)
    
    # Receita por mês (considerando meses declarados)
    df['receita_por_mes_pgdas'] = df['receita_12m'] / df['qtd_meses_declarados'].replace(0, 1)
    df['receita_por_mes_dime'] = df['receita_bruta_dime'] / df['qtd_meses_dime'].replace(0, 1)
    
    # Coeficiente de variação da receita PGDAS
    df['coef_variacao_receita'] = df['receita_desvio_padrao'] / df['receita_media'].replace(0, 1)
    
    # =========================================================================
    # GRUPO 2: FEATURES DE OMISSÃO
    # =========================================================================
    
    # Taxa de meses zerados
    df['taxa_meses_zerados_pgdas'] = df['meses_zerados_pgdas'] / df['qtd_meses_declarados'].replace(0, 1)
    df['taxa_meses_zerados_dime'] = df['meses_zerados_dime'] / df['qtd_meses_dime'].replace(0, 1)
    df['taxa_meses_sem_movimento_dime'] = df['meses_sem_movimento_dime'] / df['qtd_meses_dime'].replace(0, 1)
    
    # Flags de omissão (baseado nos indícios)
    df['omissao_pgdas'] = df['flag_omissao_pgdas']
    df['omissao_dime'] = df['flag_omissao_dime']
    
    # =========================================================================
    # GRUPO 3: FEATURES DE PAGAMENTO ICMS
    # =========================================================================
    
    df['tem_pagamentos_icms'] = (df['qtd_pagamentos_icms'] > 0).astype(int)
    df['pagamento_por_mes'] = df['valor_total_pagamentos'] / df['meses_com_pagamento'].replace(0, 1)
    
    # Ratio receita vs pagamento
    df['ratio_receita_pagamento'] = df['receita_total_consolidada'] / df['valor_total_pagamentos'].replace(0, 1)
    df['ratio_pagamento_receita'] = df['valor_total_pagamentos'] / df['receita_total_consolidada'].replace(0, 1)
    
    # Débito a recolher (DIME)
    df['tem_debito_recolher'] = (df['debito_recolher_dime'] > 0).astype(int)
    df['ratio_debito_credito_dime'] = df['total_debitos_dime'] / df['total_creditos_dime'].replace(0, 1)
    
    # =========================================================================
    # GRUPO 4: FEATURES DIMP (MEIOS DE PAGAMENTO)
    # =========================================================================
    
    df['tem_meios_pagamento'] = (df['total_meios_pagamento'] > 0).astype(int)
    
    # Proporção de cada meio
    df['proporcao_pix'] = df['total_pix'] / df['total_meios_pagamento'].replace(0, 1)
    df['proporcao_cartao_credito'] = df['total_credito_cartao'] / df['total_meios_pagamento'].replace(0, 1)
    df['proporcao_cartao_debito'] = df['total_debito_cartao'] / df['total_meios_pagamento'].replace(0, 1)
    
    # Divergência DIMP vs Receita declarada
    df['divergencia_dimp_receita'] = df['total_meios_pagamento'] - df['receita_total_consolidada']
    df['ratio_dimp_receita'] = df['total_meios_pagamento'] / df['receita_total_consolidada'].replace(0, 1)
    df['flag_dimp_maior_receita'] = (df['total_meios_pagamento'] > df['receita_total_consolidada'] * 1.1).astype(int)
    
    # =========================================================================
    # GRUPO 5: FEATURES NF-e
    # =========================================================================
    
    df['tem_nfes_emitidas'] = (df['qtd_nfes_emitidas'] > 0).astype(int)
    df['tem_nfes_recebidas'] = (df['qtd_nfes_recebidas'] > 0).astype(int)
    
    # Ratio emitidas/recebidas
    df['ratio_nfes_emit_receb'] = df['qtd_nfes_emitidas'] / df['qtd_nfes_recebidas'].replace(0, 1)
    
    # Divergência NF-e vs Receita
    df['divergencia_nfe_receita'] = df['valor_total_nfes'] - df['receita_total_consolidada']
    df['ratio_nfe_receita'] = df['valor_total_nfes'] / df['receita_total_consolidada'].replace(0, 1)
    
    # Taxa de ICMS nas NF-e
    df['taxa_icms_nfe'] = df['icms_total_nfes'] / df['valor_total_nfes'].replace(0, 1)
    
    # Concentração de clientes/fornecedores
    df['nfe_por_destinatario'] = df['qtd_nfes_emitidas'] / df['qtd_destinatarios_distintos'].replace(0, 1)
    df['nfe_por_fornecedor'] = df['qtd_nfes_recebidas'] / df['qtd_fornecedores_distintos'].replace(0, 1)
    
    # =========================================================================
    # GRUPO 6: FEATURES SOCIETÁRIAS
    # =========================================================================
    
    df['tem_socios'] = (df['qtd_socios'] > 0).astype(int)
    df['tem_vinculos'] = (df['qt_vinculos_ativos'] > 0).astype(int)
    
    # Ratio vínculos/sócios
    df['ratio_vinculos_socios'] = df['qt_vinculos_ativos'] / df['qtd_socios'].replace(0, 1)
    
    # Concentração societária
    df['concentracao_societaria'] = df['maior_participacao']
    df['sociedade_muito_concentrada'] = (df['maior_participacao'] >= 90).astype(int)
    
    # Proporção sócios PF vs PJ
    df['proporcao_socios_pf'] = df['qtd_socios_pf'] / df['qtd_socios'].replace(0, 1)
    df['tem_socios_pj'] = (df['qtd_socios_pj'] > 0).astype(int)
    
    # =========================================================================
    # GRUPO 7: FEATURES FUNCIONÁRIOS
    # =========================================================================
    
    df['tem_funcionarios'] = (df['qtd_funcionarios'] > 0).astype(int)
    
    # Receita por funcionário
    df['receita_por_funcionario'] = df['receita_total_consolidada'] / df['qtd_funcionarios'].replace(0, 1)
    
    # Folha sobre receita
    df['folha_sobre_receita'] = df['folha_total'] / df['receita_total_consolidada'].replace(0, 1)
    
    # DIMP por funcionário (proxy de movimentação)
    df['dimp_por_funcionario'] = df['total_meios_pagamento'] / df['qtd_funcionarios'].replace(0, 1)
    
    # =========================================================================
    # GRUPO 8: FEATURES CONTAS BANCÁRIAS
    # =========================================================================
    
    df['tem_contas_bancarias'] = (df['qtd_contas_bancarias'] > 0).astype(int)
    
    # Proporção de contas ativas
    df['proporcao_contas_ativas'] = df['qtd_contas_ativas'] / df['qtd_contas_bancarias'].replace(0, 1)
    
    # Contas por responsável
    df['contas_por_responsavel'] = df['qtd_contas_bancarias'] / df['qtd_responsaveis_contas'].replace(0, 1)
    
    # Diversificação bancária
    df['diversificacao_bancaria'] = df['qtd_bancos_distintos'] / df['qtd_contas_bancarias'].replace(0, 1)
    
    # =========================================================================
    # GRUPO 9: FEATURES INDÍCIOS
    # =========================================================================
    
    df['tem_indicios'] = (df['qtd_indicios'] > 0).astype(int)
    
    # Score médio por indício
    df['score_medio_indicio'] = df['soma_score_indicios'] / df['qtd_indicios'].replace(0, 1)
    
    # Indicios por tipo (diversidade de problemas)
    df['indicios_por_tipo'] = df['qtd_indicios'] / df['qtd_tipos_indicios_distintos'].replace(0, 1)
    
    # Flag de indicios graves
    df['indicios_graves'] = df['flag_indicios_graves']
    
    # =========================================================================
    # GRUPO 10: FEATURES TEMPORAIS
    # =========================================================================
    
    # Idade da empresa (em anos)
    df['anos_desde_inicio'] = df['dias_desde_inicio_icms'] / 365.25
    df['anos_desde_constituicao'] = df['dias_desde_constituicao'] / 365.25
    
    # Empresa nova vs antiga
    df['empresa_nova'] = (df['anos_desde_inicio'] < 2).astype(int)
    df['empresa_antiga'] = (df['anos_desde_inicio'] > 10).astype(int)
    
    # =========================================================================
    # GRUPO 11: SCORES COMPOSTOS
    # =========================================================================
    
    # Score de omissão (0-100)
    df['score_omissao'] = (
        df['omissao_pgdas'] * 30 +
        df['omissao_dime'] * 30 +
        df['taxa_meses_zerados_pgdas'] * 20 +
        df['taxa_meses_zerados_dime'] * 20
    )
    
    # Score de divergência (0-100)
    df['score_divergencia'] = (
        df['flag_dimp_maior_receita'] * 50 +
        np.clip(df['ratio_dimp_receita'] - 1, 0, 1) * 50
    )
    
    # Score de estrutura (0-100)
    df['score_estrutura'] = (
        (1 - df['tem_funcionarios']) * 25 +
        (1 - df['tem_contas_bancarias']) * 25 +
        df['sociedade_muito_concentrada'] * 25 +
        df['empresa_nova'] * 25
    )
    
    # Score básico combinado
    df['score_basico'] = (
        df['tem_indicios'] * 40 +
        df['indicios_graves'] * 20 +
        df['score_omissao'] * 0.2 +
        df['score_divergencia'] * 0.2
    )
    
    # Interações importantes
    df['receita_x_indicios'] = df['receita_total_consolidada'] * df['tem_indicios']
    df['omissao_x_indicios'] = df['score_omissao'] * df['tem_indicios']
    df['divergencia_x_indicios'] = df['score_divergencia'] * df['tem_indicios']
    
    return df

# ===================================================================
# APLICAR FEATURE ENGINEERING
# ===================================================================

print("\n🔧 Aplicando feature engineering...")
df_treino = criar_features_engenheiradas(df_treino)

print(f"✅ Features engenheiradas criadas!")
print(f"   Shape: {df_treino.shape}")

# ===================================================================
# TRATAMENTO DE VALORES ESPECIAIS
# ===================================================================

print("\n🔧 Tratando valores especiais...")

# Substituir infinitos por 0
df_treino = df_treino.replace([np.inf, -np.inf], 0)

# Preencher NaN com 0 para numéricas
numeric_cols = df_treino.select_dtypes(include=[np.number]).columns
df_treino[numeric_cols] = df_treino[numeric_cols].fillna(0)

print("   ✅ Infinitos e NaN tratados")

# ===================================================================
# ENCODING DE VARIÁVEIS CATEGÓRICAS
# ===================================================================

print("\n🔧 Encoding de variáveis categóricas...")

# Regime de apuração
le_regime = LabelEncoder()
df_treino['nm_reg_apuracao'] = df_treino['nm_reg_apuracao'].fillna('DESCONHECIDO')
df_treino['regime_encoded'] = le_regime.fit_transform(df_treino['nm_reg_apuracao'])
print(f"   ✅ nm_reg_apuracao: {len(le_regime.classes_)} classes")

# UF
le_uf = LabelEncoder()
df_treino['cd_uf'] = df_treino['cd_uf'].fillna('XX')
df_treino['uf_encoded'] = le_uf.fit_transform(df_treino['cd_uf'])
print(f"   ✅ cd_uf: {len(le_uf.classes_)} classes")

# CNAE Seção (primeiro dígito)
df_treino['cd_cnae'] = df_treino['cd_cnae'].fillna('0000000')
df_treino['cnae_secao'] = df_treino['cd_cnae'].astype(str).str[:2]
le_cnae = LabelEncoder()
df_treino['cnae_encoded'] = le_cnae.fit_transform(df_treino['cnae_secao'])
print(f"   ✅ cnae_secao: {len(le_cnae.classes_)} classes")

# ===================================================================
# DEFINIR LISTAS DE FEATURES
# ===================================================================

# Features básicas numéricas (da query)
numeric_features_basicas = [
    # Indícios
    'qtd_indicios', 'soma_score_indicios', 'qtd_tipos_indicios_distintos',
    'flag_omissao_dime', 'flag_omissao_pgdas', 'flag_indicios_graves',
    # PGDAS
    'receita_12m', 'receita_maxima', 'receita_minima', 'receita_media',
    'receita_desvio_padrao', 'qtd_meses_declarados', 'meses_zerados_pgdas',
    # DIME
    'receita_bruta_dime', 'faturamento_dime', 'total_creditos_dime',
    'total_debitos_dime', 'debito_recolher_dime', 'qtd_meses_dime',
    'meses_zerados_dime', 'meses_sem_movimento_dime',
    # Pagamentos
    'qtd_pagamentos_icms', 'valor_total_pagamentos', 'valor_medio_pagamento',
    'maior_pagamento', 'meses_com_pagamento',
    # DIMP
    'total_credito_cartao', 'total_debito_cartao', 'total_pix',
    'total_meios_pagamento', 'meses_com_meios_pagamento',
    # NF-e
    'qtd_nfes_emitidas', 'valor_total_nfes', 'icms_total_nfes',
    'qtd_destinatarios_distintos', 'valor_medio_nfe',
    'qtd_nfes_recebidas', 'valor_total_nfes_recebidas', 'qtd_fornecedores_distintos',
    # Sócios
    'qtd_socios', 'qtd_socios_ativos_vinculo', 'qtd_tipo_socio',
    'qtd_representantes', 'maior_participacao', 'menor_participacao',
    'media_participacao', 'qtd_socios_pf', 'qtd_socios_pj',
    # Funcionários
    'qtd_funcionarios', 'salario_medio', 'folha_total', 'maior_salario', 'menor_salario',
    # Contas
    'qtd_contas_bancarias', 'qtd_bancos_distintos', 'qtd_responsaveis_contas',
    'qtd_contas_ativas', 'qtd_contas_encerradas',
    # Cadastro
    'qt_socios_ativos', 'qt_vinculos_ativos', 'dias_desde_inicio_icms',
    'dias_desde_constituicao', 'flag_regime_normal', 'flag_simples', 'flag_uf_sc'
]

# Features engenheiradas
engineered_features = [
    # Receita
    'receita_total_consolidada', 'tem_receita', 'tem_receita_pgdas', 'tem_receita_dime',
    'receita_por_mes_pgdas', 'receita_por_mes_dime', 'coef_variacao_receita',
    # Omissão
    'taxa_meses_zerados_pgdas', 'taxa_meses_zerados_dime', 'taxa_meses_sem_movimento_dime',
    'omissao_pgdas', 'omissao_dime',
    # Pagamento
    'tem_pagamentos_icms', 'pagamento_por_mes', 'ratio_receita_pagamento',
    'ratio_pagamento_receita', 'tem_debito_recolher', 'ratio_debito_credito_dime',
    # DIMP
    'tem_meios_pagamento', 'proporcao_pix', 'proporcao_cartao_credito',
    'proporcao_cartao_debito', 'divergencia_dimp_receita', 'ratio_dimp_receita',
    'flag_dimp_maior_receita',
    # NF-e
    'tem_nfes_emitidas', 'tem_nfes_recebidas', 'ratio_nfes_emit_receb',
    'divergencia_nfe_receita', 'ratio_nfe_receita', 'taxa_icms_nfe',
    'nfe_por_destinatario', 'nfe_por_fornecedor',
    # Sócios
    'tem_socios', 'tem_vinculos', 'ratio_vinculos_socios',
    'concentracao_societaria', 'sociedade_muito_concentrada',
    'proporcao_socios_pf', 'tem_socios_pj',
    # Funcionários
    'tem_funcionarios', 'receita_por_funcionario', 'folha_sobre_receita', 'dimp_por_funcionario',
    # Contas
    'tem_contas_bancarias', 'proporcao_contas_ativas', 'contas_por_responsavel', 'diversificacao_bancaria',
    # Indícios
    'tem_indicios', 'score_medio_indicio', 'indicios_por_tipo', 'indicios_graves',
    # Temporal
    'anos_desde_inicio', 'anos_desde_constituicao', 'empresa_nova', 'empresa_antiga',
    # Scores
    'score_omissao', 'score_divergencia', 'score_estrutura', 'score_basico',
    'receita_x_indicios', 'omissao_x_indicios', 'divergencia_x_indicios'
]

# Features categóricas (encoded)
categorical_features = ['regime_encoded', 'uf_encoded', 'cnae_encoded']

# Todas as features para o modelo
all_features = numeric_features_basicas + engineered_features + categorical_features

# Filtrar apenas features que existem
all_features = [f for f in all_features if f in df_treino.columns]

print(f"\n📊 Resumo de features:")
print(f"   Features básicas: {len([f for f in numeric_features_basicas if f in df_treino.columns])}")
print(f"   Features engenheiradas: {len([f for f in engineered_features if f in df_treino.columns])}")
print(f"   Features categóricas: {len(categorical_features)}")
print(f"   Total para modelo: {len(all_features)}")

# ===================================================================
# SALVAR DADOS E ENCODERS
# ===================================================================

print("\n💾 Salvando dados e encoders...")

import pickle

# Salvar encoders
with open('le_regime.pkl', 'wb') as f:
    pickle.dump(le_regime, f)
with open('le_uf.pkl', 'wb') as f:
    pickle.dump(le_uf, f)
with open('le_cnae.pkl', 'wb') as f:
    pickle.dump(le_cnae, f)
with open('all_features.pkl', 'wb') as f:
    pickle.dump(all_features, f)

# Salvar dataframe processado
df_treino.to_csv('df_treino_processado.csv', index=False)

print("   ✅ le_regime.pkl")
print("   ✅ le_uf.pkl")
print("   ✅ le_cnae.pkl")
print("   ✅ all_features.pkl")
print("   ✅ df_treino_processado.csv")

print("\n" + "="*80)
print("✅ PASSO 2B CONCLUÍDO!")
print("="*80)

🔧 PASSO 2B: CRIANDO FEATURES ENGENHEIRADAS

🔧 Convertendo tipos Decimal para float...
   Convertido: receita_12m
   Convertido: receita_maxima
   Convertido: receita_minima
   Convertido: receita_media
   Convertido: receita_bruta_dime
   Convertido: faturamento_dime
   Convertido: total_creditos_dime
   Convertido: total_debitos_dime
   Convertido: debito_recolher_dime
   Convertido: cred_mes_anterior_dime
   Convertido: cred_mes_seguinte_dime
   Convertido: valor_total_pagamentos
   Convertido: valor_medio_pagamento
   Convertido: maior_pagamento
   Convertido: total_credito_cartao
   Convertido: total_debito_cartao
   Convertido: total_pix
   Convertido: total_meios_pagamento
   Convertido: valor_total_nfes
   Convertido: icms_total_nfes
   Convertido: valor_medio_nfe
   Convertido: valor_total_nfes_recebidas
   Convertido: maior_participacao
   Convertido: menor_participacao
   Convertido: media_participacao
   Convertido: salario_medio
   Convertido: folha_total
   Convertido: mai

In [9]:
# ==============================================================================
# CÉLULA 3: TREINO DO MODELO
# ==============================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import pickle

print("="*80)
print("🤖 PASSO 3: TREINANDO MODELO")
print("="*80)

# ===================================================================
# CARREGAR DADOS E FEATURES
# ===================================================================

print("\n📊 Carregando dados...")

# Carregar lista de features
with open('all_features.pkl', 'rb') as f:
    all_features = pickle.load(f)

print(f"   Features para modelo: {len(all_features)}")

# ===================================================================
# PREPARAR X e y
# ===================================================================

print("\n🔧 Preparando dados para treino...")

# Features (X)
X = df_treino[all_features].copy()

# Target (y)
y = df_treino['target_deve_cancelar'].copy()

print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Target distribution:")
print(f"      0 (não cancelar): {(y == 0).sum():,}")
print(f"      1 (deve cancelar): {(y == 1).sum():,}")

# ===================================================================
# SPLIT TREINO/TESTE
# ===================================================================

print("\n🔧 Dividindo em treino/teste...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"   Treino: {len(X_train):,} registros")
print(f"   Teste: {len(X_test):,} registros")

# ===================================================================
# TREINAR MODELO
# ===================================================================

print("\n🤖 Treinando Random Forest...")

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train, y_train)
print("   ✅ Modelo treinado!")

# ===================================================================
# AVALIAR MODELO
# ===================================================================

print("\n📊 Avaliando modelo...")

# Predições
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\n🎯 ROC AUC Score: {roc_auc:.4f}")

# Confusion Matrix
print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"   TN: {cm[0,0]:,}  FP: {cm[0,1]:,}")
print(f"   FN: {cm[1,0]:,}  TP: {cm[1,1]:,}")

# ===================================================================
# FEATURE IMPORTANCE
# ===================================================================

print("\n📊 Top 20 Features Mais Importantes:")

feature_importance = pd.DataFrame({
    'feature': all_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

for i, row in feature_importance.head(20).iterrows():
    print(f"   {row['feature']}: {row['importance']:.4f}")

# ===================================================================
# ANÁLISE DE DIFERENTES LIMIARES
# ===================================================================

print("\n" + "="*80)
print("🎯 ANÁLISE DE LIMIARES DE CLASSIFICAÇÃO")
print("="*80)

from sklearn.metrics import precision_score, recall_score, f1_score
import builtins  # Para usar o round nativo do Python

limiares = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
resultados_limiares = []

for limiar in limiares:
    y_pred_limiar = (y_pred_proba >= limiar).astype(int)
    
    precision = precision_score(y_test, y_pred_limiar, zero_division=0)
    recall = recall_score(y_test, y_pred_limiar, zero_division=0)
    f1 = f1_score(y_test, y_pred_limiar, zero_division=0)
    
    n_alto_risco = y_pred_limiar.sum()
    
    resultados_limiares.append({
        'limiar': limiar,
        'precision': builtins.round(float(precision), 4),
        'recall': builtins.round(float(recall), 4),
        'f1_score': builtins.round(float(f1), 4),
        'n_alto_risco': int(n_alto_risco),
        'perc_alto_risco': builtins.round(float(n_alto_risco) / len(y_test) * 100, 2)
    })

df_limiares = pd.DataFrame(resultados_limiares)
print("\n📊 Resultados por limiar:")
print(df_limiares.to_string(index=False))

# Salvar análise de limiares
df_limiares.to_csv('analise_limiares.csv', index=False)
print("\n   ✅ analise_limiares.csv")

# ===================================================================
# SALVAR MODELO E ARTEFATOS
# ===================================================================

print("\n" + "="*80)
print("💾 SALVANDO MODELO E ARTEFATOS")
print("="*80)

# Salvar modelo
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
print("   ✅ rf_model.pkl")

# Salvar feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print("   ✅ feature_importance.csv")

# Salvar dados de treino final
df_treino.to_csv('df_treino_final.csv', index=False)
print("   ✅ df_treino_final.csv")

# Salvar dataset completo para referência
df_treino[['cnpj', 'target_deve_cancelar'] + all_features].to_csv(
    'df_treino_completo.csv', 
    index=False
)
print("   ✅ df_treino_completo.csv")

print("\n" + "="*80)
print("✅ PASSO 3 CONCLUÍDO!")
print(f"   ROC AUC: {roc_auc:.4f}")
print("="*80)

🤖 PASSO 3: TREINANDO MODELO

📊 Carregando dados...
   Features para modelo: 131

🔧 Preparando dados para treino...
   X shape: (293053, 131)
   y shape: (293053,)
   Target distribution:
      0 (não cancelar): 60,622
      1 (deve cancelar): 232,431

🔧 Dividindo em treino/teste...
   Treino: 234,442 registros
   Teste: 58,611 registros

🤖 Treinando Random Forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 128 concurrent workers.
[Parallel(n_jobs=-1)]: Done  47 out of 100 | elapsed:    1.3s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=100)]: Using backend ThreadingBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   2 out of 100 | elapsed:    0.0s remaining:    1.1s
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=100)]: Using backend ThreadingBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   2 out of 100 | elapsed:    0.0s remaining:    1.0s
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    0.0s finished


   ✅ Modelo treinado!

📊 Avaliando modelo...

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     12124
           1       1.00      0.99      1.00     46487

    accuracy                           0.99     58611
   macro avg       0.99      0.99      0.99     58611
weighted avg       0.99      0.99      0.99     58611


🎯 ROC AUC Score: 0.9995

📊 Confusion Matrix:
   TN: 12,012  FP: 112
   FN: 236  TP: 46,251

📊 Top 20 Features Mais Importantes:
   qtd_tipos_indicios_distintos: 0.1580
   soma_score_indicios: 0.1542
   qtd_indicios: 0.1412
   score_medio_indicio: 0.1238
   tem_indicios: 0.0921
   score_basico: 0.0752
   indicios_por_tipo: 0.0658
   receita_x_indicios: 0.0248
   valor_total_nfes_recebidas: 0.0156
   divergencia_x_indicios: 0.0140
   qtd_nfes_recebidas: 0.0106
   ratio_nfe_receita: 0.0093
   valor_medio_nfe: 0.0083
   omissao_x_indicios: 0.0077
   qtd_nfes_emitidas: 0.0056
   total_meios_pagament

In [6]:
# ==============================================================================
# CÉLULA 4: BUSCAR FEATURES PARA EMPRESAS ATIVAS (CORRIGIDA)
# ==============================================================================
# Versão corrigida que:
# - Usa ods_pagamento_raw em vez de vw_ods_pagamento
# - Usa cad_vinculo_raw em vez de vw_cad_vinculo
# ==============================================================================

import pandas as pd
import numpy as np
from datetime import datetime

print("="*80)
print("📊 PASSO 4: BUSCANDO FEATURES PARA EMPRESAS ATIVAS")
print("="*80)
print(f"Início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ===================================================================
# BUSCAR CNPJs ATIVOS PARA PREDIÇÃO
# ===================================================================

print("\n🔍 Buscando empresas ativas para predição...")

query_cnpjs_ativos = """
SELECT DISTINCT
    c.nu_cnpj AS cnpj
FROM usr_sat_ods.vw_ods_contrib c
WHERE c.nm_sit_cadastral = 'ATIVO'
    AND c.cd_uf = 'SC'
ORDER BY 1
LIMIT 400000
"""

df_cnpjs_ativos = spark.sql(query_cnpjs_ativos).toPandas()
print(f"✅ {len(df_cnpjs_ativos):,} empresas ativas encontradas")

cnpjs_pred = df_cnpjs_ativos['cnpj'].tolist()

# ===================================================================
# QUERY DE FEATURES (MESMA DO TREINO - CORRIGIDA)
# ===================================================================

def criar_filtro_cnpjs(cnpjs, batch_size=10000):
    batches = [cnpjs[i:i + batch_size] for i in range(0, len(cnpjs), batch_size)]
    return batches

batches = criar_filtro_cnpjs(cnpjs_pred, batch_size=10000)
print(f"📦 Dividindo {len(cnpjs_pred):,} CNPJs em {len(batches)} batches")

query_features_pred = """
WITH cnpjs_pred AS (
    SELECT cnpj FROM (
        VALUES {cnpjs_values}
    ) AS t(cnpj)
),

-- ============================================================================
-- 1. INDÍCIOS NEAF
-- ============================================================================
indicios AS (
    SELECT
        ind.nu_cpf_cnpj AS cnpj,
        COUNT(*) AS qtd_indicios,
        SUM(COALESCE(ind.nu_score, 0)) AS soma_score_indicios,
        COUNT(DISTINCT ind.cd_indicio) AS qtd_tipos_indicios_distintos,
        MAX(CASE WHEN ind.cd_indicio = 11 THEN 1 ELSE 0 END) AS flag_omissao_dime,
        MAX(CASE WHEN ind.cd_indicio = 12 THEN 1 ELSE 0 END) AS flag_omissao_pgdas,
        MAX(CASE WHEN ind.cd_indicio IN (1,2,3,4,5) THEN 1 ELSE 0 END) AS flag_indicios_graves
    FROM neaf.empresa_indicio ind
    WHERE ind.nu_cpf_cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND ind.cd_atual = 1
    GROUP BY ind.nu_cpf_cnpj
),

-- ============================================================================
-- 2. PGDAS
-- ============================================================================
pgdas AS (
    SELECT
        p.nu_cnpj AS cnpj,
        SUM(p.vl_rec_bruta_estab) AS receita_12m,
        MAX(p.vl_rec_bruta_estab) AS receita_maxima,
        MIN(CASE WHEN p.vl_rec_bruta_estab > 0 THEN p.vl_rec_bruta_estab ELSE NULL END) AS receita_minima,
        AVG(p.vl_rec_bruta_estab) AS receita_media,
        STDDEV(p.vl_rec_bruta_estab) AS receita_desvio_padrao,
        COUNT(DISTINCT p.nu_per_ref) AS qtd_meses_declarados,
        SUM(CASE WHEN p.vl_rec_bruta_estab = 0 THEN 1 ELSE 0 END) AS meses_zerados_pgdas,
        MAX(p.nu_per_ref) AS ultimo_periodo_pgdas,
        MIN(p.nu_per_ref) AS primeiro_periodo_pgdas
    FROM usr_sat_ods.sna_pgdasd_estabelecimento_raw p
    WHERE p.nu_cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND p.nu_per_ref >= 202301
    GROUP BY p.nu_cnpj
),

-- ============================================================================
-- 3. DIME
-- ============================================================================
dime AS (
    SELECT
        d.nu_cnpj AS cnpj,
        SUM(d.vl_receita_bruta) AS receita_bruta_dime,
        SUM(d.vl_faturamento) AS faturamento_dime,
        SUM(d.vl_tot_cred) AS total_creditos_dime,
        SUM(d.vl_tot_deb) AS total_debitos_dime,
        SUM(d.vl_deb_recolher) AS debito_recolher_dime,
        SUM(d.vl_cred_mes_anterior) AS cred_mes_anterior_dime,
        SUM(d.vl_cred_mes_seguinte) AS cred_mes_seguinte_dime,
        COUNT(DISTINCT d.nu_per_ref) AS qtd_meses_dime,
        SUM(CASE WHEN d.vl_faturamento = 0 THEN 1 ELSE 0 END) AS meses_zerados_dime,
        SUM(CASE WHEN d.vl_tot_cred = 0 AND d.vl_tot_deb = 0 THEN 1 ELSE 0 END) AS meses_sem_movimento_dime,
        MAX(d.nu_per_ref) AS ultimo_periodo_dime
    FROM usr_sat_ods.ods_decl_dime_raw d
    WHERE d.nu_cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND d.nu_per_ref >= 202301
    GROUP BY d.nu_cnpj
),

-- ============================================================================
-- 4. RECOLHIMENTO (CORRIGIDO - usa tabela RAW)
-- ============================================================================
recolhimento AS (
    SELECT
        p.nu_cnpj AS cnpj,
        COUNT(p.nu_id_pagamento) AS qtd_pagamentos_icms,
        SUM(p.vl_total) AS valor_total_pagamentos,
        AVG(p.vl_total) AS valor_medio_pagamento,
        MAX(p.vl_total) AS maior_pagamento,
        COUNT(DISTINCT p.nu_per_pagamento) AS meses_com_pagamento,
        MAX(p.nu_per_pagamento) AS ultimo_periodo_pagamento
    FROM usr_sat_ods.ods_pagamento_raw p
    WHERE p.nu_cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND p.nu_per_pagamento >= 202301
        AND p.cd_receita IN (1449, 1465, 1473, 1554, 1570, 1589, 1597, 1600, 1643, 1651, 
                             1716, 1724, 1732, 1740, 1759, 1767, 1783, 1791, 2140, 2496, 
                             2526, 2534, 3000, 3050, 3662, 4383, 7110, 7137, 9687, 9784)
    GROUP BY p.nu_cnpj
),

-- ============================================================================
-- 5. DIMP
-- ============================================================================
dimp AS (
    SELECT
        ato_nu_cnpjmf AS cnpj,
        SUM(ato_vl_credito) AS total_credito_cartao,
        SUM(ato_vl_debito) AS total_debito_cartao,
        SUM(ato_vl_pix) AS total_pix,
        SUM(ato_vl_credito + ato_vl_debito + ato_vl_pix) AS total_meios_pagamento,
        COUNT(DISTINCT ato_dt_referencia) AS meses_com_meios_pagamento
    FROM usr_sat_admcc.acc_r66_totalestab
    WHERE ato_nu_cnpjmf IN (SELECT cnpj FROM cnpjs_pred)
        AND ato_dt_referencia >= 202301
        AND LENGTH(TRIM(ato_nu_cnpjmf)) = 14
    GROUP BY ato_nu_cnpjmf
),

-- ============================================================================
-- 6. NF-e EMITIDAS
-- ============================================================================
nfe_emit AS (
    SELECT
        procnfe.nfe.infnfe.emit.cnpj AS cnpj,
        COUNT(DISTINCT chave) AS qtd_nfes_emitidas,
        SUM(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_total_nfes,
        SUM(procnfe.nfe.infnfe.total.icmstot.vicms) AS icms_total_nfes,
        COUNT(DISTINCT procnfe.nfe.infnfe.dest.cnpj) AS qtd_destinatarios_distintos,
        AVG(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_medio_nfe
    FROM nfe.nfe
    WHERE procnfe.nfe.infnfe.emit.cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND (ano_emissao * 100 + mes_emissao) >= 202301
        AND procnfe.nfe.infnfe.ide.tpnf = 1
    GROUP BY procnfe.nfe.infnfe.emit.cnpj
),

-- ============================================================================
-- 7. NF-e RECEBIDAS
-- ============================================================================
nfe_receb AS (
    SELECT
        procnfe.nfe.infnfe.dest.cnpj AS cnpj,
        COUNT(DISTINCT chave) AS qtd_nfes_recebidas,
        SUM(procnfe.nfe.infnfe.total.icmstot.vnf) AS valor_total_nfes_recebidas,
        COUNT(DISTINCT procnfe.nfe.infnfe.emit.cnpj) AS qtd_fornecedores_distintos
    FROM nfe.nfe
    WHERE procnfe.nfe.infnfe.dest.cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND (ano_emissao * 100 + mes_emissao) >= 202301
    GROUP BY procnfe.nfe.infnfe.dest.cnpj
),

-- ============================================================================
-- 8. VÍNCULOS SOCIETÁRIOS (CORRIGIDO - usa tabela RAW)
-- ============================================================================
socios AS (
    SELECT
        v.nu_cnpj_princ AS cnpj,
        COUNT(DISTINCT COALESCE(v.nu_cnpj_secund, v.nu_cpf_secund)) AS qtd_socios,
        COUNT(DISTINCT CASE WHEN v.sn_relacao_ativa = 1 THEN COALESCE(v.nu_cnpj_secund, v.nu_cpf_secund) END) AS qtd_socios_ativos_vinculo,
        SUM(CASE WHEN v.nm_relacao = 'SOCIO' OR v.nm_relacao = 'SÓCIO' THEN 1 ELSE 0 END) AS qtd_tipo_socio,
        SUM(CASE WHEN v.nm_relacao = 'REPRESENTANTE' THEN 1 ELSE 0 END) AS qtd_representantes,
        MAX(v.pe_capital_empresa) AS maior_participacao,
        MIN(CASE WHEN v.pe_capital_empresa > 0 THEN v.pe_capital_empresa ELSE NULL END) AS menor_participacao,
        AVG(v.pe_capital_empresa) AS media_participacao,
        COUNT(DISTINCT CASE WHEN v.nu_cpf_secund IS NOT NULL THEN v.nu_cpf_secund END) AS qtd_socios_pf,
        COUNT(DISTINCT CASE WHEN v.nu_cnpj_secund IS NOT NULL THEN v.nu_cnpj_secund END) AS qtd_socios_pj
    FROM usr_sat_ods.cad_vinculo_raw v
    WHERE v.nu_cnpj_princ IN (SELECT cnpj FROM cnpjs_pred)
        AND v.nm_relacao NOT IN ('CONTABILISTA')
    GROUP BY v.nu_cnpj_princ
),

-- ============================================================================
-- 9. FUNCIONÁRIOS RAIS
-- ============================================================================
funcionarios AS (
    SELECT
        r.cnpj_cei AS cnpj,
        COUNT(DISTINCT r.cpf) AS qtd_funcionarios,
        AVG(r.vl_remun_media_nom) AS salario_medio,
        SUM(r.vl_remun_media_nom) AS folha_total,
        MAX(r.vl_remun_media_nom) AS maior_salario,
        MIN(CASE WHEN r.vl_remun_media_nom > 0 THEN r.vl_remun_media_nom ELSE NULL END) AS menor_salario
    FROM rais_caged.vw_rais_vinculos r
    WHERE r.cnpj_cei IN (SELECT cnpj FROM cnpjs_pred)
        AND r.vinculo_ativo_31_12 = 1
        AND r.ano_arquivo >= 2023
    GROUP BY r.cnpj_cei
),

-- ============================================================================
-- 10. CONTAS BANCÁRIAS
-- ============================================================================
contas_bancarias AS (
    SELECT
        c.nr_cnpj AS cnpj,
        COUNT(*) AS qtd_contas_bancarias,
        COUNT(DISTINCT c.nm_banco) AS qtd_bancos_distintos,
        COUNT(DISTINCT c.nr_cpf) AS qtd_responsaveis_contas,
        SUM(CASE WHEN c.dt_encerramento IS NULL THEN 1 ELSE 0 END) AS qtd_contas_ativas,
        SUM(CASE WHEN c.dt_encerramento IS NOT NULL THEN 1 ELSE 0 END) AS qtd_contas_encerradas
    FROM usr_sat_fsn.fsn_conta_bancaria c
    WHERE c.nr_cnpj IN (SELECT cnpj FROM cnpjs_pred)
        AND (c.valido IS NULL OR c.valido = 1)
    GROUP BY c.nr_cnpj
),

-- ============================================================================
-- 11. CADASTRO
-- ============================================================================
cadastro AS (
    SELECT
        c.nu_cnpj AS cnpj,
        c.cd_cnae,
        c.nm_reg_apuracao,
        c.cd_uf,
        c.qt_socios_ativos,
        c.qt_vinculos_ativos,
        DATEDIFF(CURRENT_DATE(), c.dt_inicio_icms) AS dias_desde_inicio_icms,
        DATEDIFF(CURRENT_DATE(), c.dt_constituicao_empresa) AS dias_desde_constituicao,
        CASE WHEN c.nm_reg_apuracao = 'NORMAL' THEN 1 ELSE 0 END AS flag_regime_normal,
        CASE WHEN c.nm_reg_apuracao IN ('SIMPLES NACIONAL', 'SIMEI', 'SIMPLES SC') THEN 1 ELSE 0 END AS flag_simples,
        CASE WHEN c.cd_uf = 'SC' THEN 1 ELSE 0 END AS flag_uf_sc
    FROM usr_sat_ods.vw_ods_contrib c
    WHERE c.nu_cnpj IN (SELECT cnpj FROM cnpjs_pred)
)

-- ============================================================================
-- CONSOLIDAÇÃO FINAL
-- ============================================================================
SELECT
    ct.cnpj,
    
    -- INDÍCIOS
    COALESCE(ind.qtd_indicios, 0) AS qtd_indicios,
    COALESCE(ind.soma_score_indicios, 0) AS soma_score_indicios,
    COALESCE(ind.qtd_tipos_indicios_distintos, 0) AS qtd_tipos_indicios_distintos,
    COALESCE(ind.flag_omissao_dime, 0) AS flag_omissao_dime,
    COALESCE(ind.flag_omissao_pgdas, 0) AS flag_omissao_pgdas,
    COALESCE(ind.flag_indicios_graves, 0) AS flag_indicios_graves,
    
    -- PGDAS
    COALESCE(pg.receita_12m, 0) AS receita_12m,
    COALESCE(pg.receita_maxima, 0) AS receita_maxima,
    COALESCE(pg.receita_minima, 0) AS receita_minima,
    COALESCE(pg.receita_media, 0) AS receita_media,
    COALESCE(pg.receita_desvio_padrao, 0) AS receita_desvio_padrao,
    COALESCE(pg.qtd_meses_declarados, 0) AS qtd_meses_declarados,
    COALESCE(pg.meses_zerados_pgdas, 0) AS meses_zerados_pgdas,
    COALESCE(pg.ultimo_periodo_pgdas, 0) AS ultimo_periodo_pgdas,
    COALESCE(pg.primeiro_periodo_pgdas, 0) AS primeiro_periodo_pgdas,
    
    -- DIME
    COALESCE(dm.receita_bruta_dime, 0) AS receita_bruta_dime,
    COALESCE(dm.faturamento_dime, 0) AS faturamento_dime,
    COALESCE(dm.total_creditos_dime, 0) AS total_creditos_dime,
    COALESCE(dm.total_debitos_dime, 0) AS total_debitos_dime,
    COALESCE(dm.debito_recolher_dime, 0) AS debito_recolher_dime,
    COALESCE(dm.cred_mes_anterior_dime, 0) AS cred_mes_anterior_dime,
    COALESCE(dm.cred_mes_seguinte_dime, 0) AS cred_mes_seguinte_dime,
    COALESCE(dm.qtd_meses_dime, 0) AS qtd_meses_dime,
    COALESCE(dm.meses_zerados_dime, 0) AS meses_zerados_dime,
    COALESCE(dm.meses_sem_movimento_dime, 0) AS meses_sem_movimento_dime,
    COALESCE(dm.ultimo_periodo_dime, 0) AS ultimo_periodo_dime,
    
    -- RECOLHIMENTO
    COALESCE(rec.qtd_pagamentos_icms, 0) AS qtd_pagamentos_icms,
    COALESCE(rec.valor_total_pagamentos, 0) AS valor_total_pagamentos,
    COALESCE(rec.valor_medio_pagamento, 0) AS valor_medio_pagamento,
    COALESCE(rec.maior_pagamento, 0) AS maior_pagamento,
    COALESCE(rec.meses_com_pagamento, 0) AS meses_com_pagamento,
    COALESCE(rec.ultimo_periodo_pagamento, 0) AS ultimo_periodo_pagamento,
    
    -- DIMP
    COALESCE(dp.total_credito_cartao, 0) AS total_credito_cartao,
    COALESCE(dp.total_debito_cartao, 0) AS total_debito_cartao,
    COALESCE(dp.total_pix, 0) AS total_pix,
    COALESCE(dp.total_meios_pagamento, 0) AS total_meios_pagamento,
    COALESCE(dp.meses_com_meios_pagamento, 0) AS meses_com_meios_pagamento,
    
    -- NF-e EMITIDAS
    COALESCE(ne.qtd_nfes_emitidas, 0) AS qtd_nfes_emitidas,
    COALESCE(ne.valor_total_nfes, 0) AS valor_total_nfes,
    COALESCE(ne.icms_total_nfes, 0) AS icms_total_nfes,
    COALESCE(ne.qtd_destinatarios_distintos, 0) AS qtd_destinatarios_distintos,
    COALESCE(ne.valor_medio_nfe, 0) AS valor_medio_nfe,
    
    -- NF-e RECEBIDAS
    COALESCE(nr.qtd_nfes_recebidas, 0) AS qtd_nfes_recebidas,
    COALESCE(nr.valor_total_nfes_recebidas, 0) AS valor_total_nfes_recebidas,
    COALESCE(nr.qtd_fornecedores_distintos, 0) AS qtd_fornecedores_distintos,
    
    -- SÓCIOS
    COALESCE(soc.qtd_socios, 0) AS qtd_socios,
    COALESCE(soc.qtd_socios_ativos_vinculo, 0) AS qtd_socios_ativos_vinculo,
    COALESCE(soc.qtd_tipo_socio, 0) AS qtd_tipo_socio,
    COALESCE(soc.qtd_representantes, 0) AS qtd_representantes,
    COALESCE(soc.maior_participacao, 0) AS maior_participacao,
    COALESCE(soc.menor_participacao, 0) AS menor_participacao,
    COALESCE(soc.media_participacao, 0) AS media_participacao,
    COALESCE(soc.qtd_socios_pf, 0) AS qtd_socios_pf,
    COALESCE(soc.qtd_socios_pj, 0) AS qtd_socios_pj,
    
    -- FUNCIONÁRIOS
    COALESCE(func.qtd_funcionarios, 0) AS qtd_funcionarios,
    COALESCE(func.salario_medio, 0) AS salario_medio,
    COALESCE(func.folha_total, 0) AS folha_total,
    COALESCE(func.maior_salario, 0) AS maior_salario,
    COALESCE(func.menor_salario, 0) AS menor_salario,
    
    -- CONTAS BANCÁRIAS
    COALESCE(cb.qtd_contas_bancarias, 0) AS qtd_contas_bancarias,
    COALESCE(cb.qtd_bancos_distintos, 0) AS qtd_bancos_distintos,
    COALESCE(cb.qtd_responsaveis_contas, 0) AS qtd_responsaveis_contas,
    COALESCE(cb.qtd_contas_ativas, 0) AS qtd_contas_ativas,
    COALESCE(cb.qtd_contas_encerradas, 0) AS qtd_contas_encerradas,
    
    -- CADASTRO
    cad.cd_cnae,
    cad.nm_reg_apuracao,
    cad.cd_uf,
    COALESCE(cad.qt_socios_ativos, 0) AS qt_socios_ativos,
    COALESCE(cad.qt_vinculos_ativos, 0) AS qt_vinculos_ativos,
    COALESCE(cad.dias_desde_inicio_icms, 0) AS dias_desde_inicio_icms,
    COALESCE(cad.dias_desde_constituicao, 0) AS dias_desde_constituicao,
    COALESCE(cad.flag_regime_normal, 0) AS flag_regime_normal,
    COALESCE(cad.flag_simples, 0) AS flag_simples,
    COALESCE(cad.flag_uf_sc, 0) AS flag_uf_sc

FROM cnpjs_pred ct
LEFT JOIN indicios ind ON ct.cnpj = ind.cnpj
LEFT JOIN pgdas pg ON ct.cnpj = pg.cnpj
LEFT JOIN dime dm ON ct.cnpj = dm.cnpj
LEFT JOIN recolhimento rec ON ct.cnpj = rec.cnpj
LEFT JOIN dimp dp ON ct.cnpj = dp.cnpj
LEFT JOIN nfe_emit ne ON ct.cnpj = ne.cnpj
LEFT JOIN nfe_receb nr ON ct.cnpj = nr.cnpj
LEFT JOIN socios soc ON ct.cnpj = soc.cnpj
LEFT JOIN funcionarios func ON ct.cnpj = func.cnpj
LEFT JOIN contas_bancarias cb ON ct.cnpj = cb.cnpj
LEFT JOIN cadastro cad ON ct.cnpj = cad.cnpj
"""

print("\n🔍 Buscando features para empresas ativas...")
print("   Fontes: Indícios, PGDAS, DIME, Pagamentos, DIMP, NF-e, Sócios, Funcionários, Contas, Cadastro")

df_pred_list = []
batches_sucesso = 0
batches_erro = 0

for i, batch in enumerate(batches, 1):
    print(f"   📦 Batch {i}/{len(batches)} ({len(batch):,} CNPJs)...", end=" ")
    cnpjs_values = ",".join([f"('{cnpj}')" for cnpj in batch])
    query = query_features_pred.replace("{cnpjs_values}", cnpjs_values)
    
    try:
        df_batch = spark.sql(query).toPandas()
        df_pred_list.append(df_batch)
        batches_sucesso += 1
        print("✅")
    except Exception as e:
        batches_erro += 1
        print(f"❌ Erro: {str(e)[:50]}...")
        continue

print(f"\n📊 Resumo:")
print(f"   ✅ Sucesso: {batches_sucesso}/{len(batches)}")
print(f"   ❌ Erro: {batches_erro}/{len(batches)}")

if df_pred_list:
    df_pred_features = pd.concat(df_pred_list, ignore_index=True)
    print(f"\n✅ Features para predição consolidadas:")
    print(f"   Registros: {len(df_pred_features):,}")
    print(f"   Colunas: {len(df_pred_features.columns)}")
    
    # Salvar checkpoint
    print(f"\n💾 Salvando checkpoint...")
    df_pred_features.to_csv('df_pred_features_raw.csv', index=False)
    print(f"   ✅ Salvo em: df_pred_features_raw.csv")
else:
    print("\n❌ ERRO: Nenhum batch processado com sucesso!")

print("\n" + "="*80)
print("✅ PASSO 4 CONCLUÍDO!")
print("="*80)

📊 PASSO 4: BUSCANDO FEATURES PARA EMPRESAS ATIVAS
Início: 2025-11-27 19:10:23

🔍 Buscando empresas ativas para predição...


Hive Session ID = 7c0644ca-e571-4f7f-9fcd-bb8c53e24d87
                                                                                

✅ 400,000 empresas ativas encontradas
📦 Dividindo 400,000 CNPJs em 40 batches

🔍 Buscando features para empresas ativas...
   Fontes: Indícios, PGDAS, DIME, Pagamentos, DIMP, NF-e, Sócios, Funcionários, Contas, Cadastro
   📦 Batch 1/40 (10,000 CNPJs)... 

                                                                                2]]]]

✅
   📦 Batch 2/40 (10,000 CNPJs)... 

...

✅
   📦 Batch 39/40 (10,000 CNPJs)... 

                                                                                ]28]172]]]]

✅
   📦 Batch 40/40 (10,000 CNPJs)... 

                                                                                29]]172]]]]

✅

📊 Resumo:
   ✅ Sucesso: 40/40
   ❌ Erro: 0/40

✅ Features para predição consolidadas:
   Registros: 400,000
   Colunas: 75

💾 Salvando checkpoint...
   ✅ Salvo em: df_pred_features_raw.csv

✅ PASSO 4 CONCLUÍDO!


In [7]:
# ==============================================================================
# CÉLULA 5: PREDIÇÃO PARA EMPRESAS ATIVAS
# ==============================================================================

import pandas as pd
import numpy as np
import pickle
from datetime import datetime

print("="*80)
print("🔮 PASSO 5: PREDIÇÃO PARA EMPRESAS ATIVAS")
print("="*80)
print(f"Início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ===================================================================
# CARREGAR MODELO E ARTEFATOS
# ===================================================================

print("\n📥 Carregando modelo e artefatos...")

with open('rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)
print("   ✅ rf_model.pkl")

with open('le_regime.pkl', 'rb') as f:
    le_regime = pickle.load(f)
print("   ✅ le_regime.pkl")

with open('le_uf.pkl', 'rb') as f:
    le_uf = pickle.load(f)
print("   ✅ le_uf.pkl")

with open('le_cnae.pkl', 'rb') as f:
    le_cnae = pickle.load(f)
print("   ✅ le_cnae.pkl")

with open('all_features.pkl', 'rb') as f:
    all_features = pickle.load(f)
print(f"   ✅ all_features.pkl ({len(all_features)} features)")

# ===================================================================
# CARREGAR DADOS DE PREDIÇÃO
# ===================================================================

print("\n📥 Carregando dados de predição...")
df_pred = pd.read_csv('df_pred_features_raw.csv')
print(f"   Registros: {len(df_pred):,}")

# ===================================================================
# CONVERSÃO DE TIPOS DECIMAL -> FLOAT
# ===================================================================

print("\n🔧 Convertendo tipos para float...")

from decimal import Decimal

# Converter colunas numéricas que podem ter problemas de tipo
numeric_cols_to_convert = [
    'receita_12m', 'receita_maxima', 'receita_minima', 'receita_media', 'receita_desvio_padrao',
    'receita_bruta_dime', 'faturamento_dime', 'total_creditos_dime', 'total_debitos_dime',
    'debito_recolher_dime', 'cred_mes_anterior_dime', 'cred_mes_seguinte_dime',
    'valor_total_pagamentos', 'valor_medio_pagamento', 'maior_pagamento',
    'total_credito_cartao', 'total_debito_cartao', 'total_pix', 'total_meios_pagamento',
    'valor_total_nfes', 'icms_total_nfes', 'valor_medio_nfe',
    'valor_total_nfes_recebidas',
    'maior_participacao', 'menor_participacao', 'media_participacao',
    'salario_medio', 'folha_total', 'maior_salario', 'menor_salario',
    'soma_score_indicios'
]

for col in numeric_cols_to_convert:
    if col in df_pred.columns:
        df_pred[col] = pd.to_numeric(df_pred[col], errors='coerce').fillna(0).astype(float)

print("   ✅ Conversão de tipos concluída")

# ===================================================================
# FUNÇÃO DE FEATURE ENGINEERING (MESMA DO TREINO)
# ===================================================================

def criar_features_engenheiradas_pred(df):
    """
    Cria features derivadas - MESMA FUNÇÃO DO TREINO
    """
    df = df.copy()
    
    # GRUPO 1: RECEITA
    df['receita_total_consolidada'] = df['receita_12m'] + df['receita_bruta_dime']
    df['tem_receita_pgdas'] = (df['receita_12m'] > 0).astype(int)
    df['tem_receita_dime'] = (df['receita_bruta_dime'] > 0).astype(int)
    df['tem_receita'] = ((df['receita_12m'] > 0) | (df['receita_bruta_dime'] > 0)).astype(int)
    df['receita_por_mes_pgdas'] = df['receita_12m'] / df['qtd_meses_declarados'].replace(0, 1)
    df['receita_por_mes_dime'] = df['receita_bruta_dime'] / df['qtd_meses_dime'].replace(0, 1)
    df['coef_variacao_receita'] = df['receita_desvio_padrao'] / df['receita_media'].replace(0, 1)
    
    # GRUPO 2: OMISSÃO
    df['taxa_meses_zerados_pgdas'] = df['meses_zerados_pgdas'] / df['qtd_meses_declarados'].replace(0, 1)
    df['taxa_meses_zerados_dime'] = df['meses_zerados_dime'] / df['qtd_meses_dime'].replace(0, 1)
    df['taxa_meses_sem_movimento_dime'] = df['meses_sem_movimento_dime'] / df['qtd_meses_dime'].replace(0, 1)
    df['omissao_pgdas'] = df['flag_omissao_pgdas']
    df['omissao_dime'] = df['flag_omissao_dime']
    
    # GRUPO 3: PAGAMENTO
    df['tem_pagamentos_icms'] = (df['qtd_pagamentos_icms'] > 0).astype(int)
    df['pagamento_por_mes'] = df['valor_total_pagamentos'] / df['meses_com_pagamento'].replace(0, 1)
    df['ratio_receita_pagamento'] = df['receita_total_consolidada'] / df['valor_total_pagamentos'].replace(0, 1)
    df['ratio_pagamento_receita'] = df['valor_total_pagamentos'] / df['receita_total_consolidada'].replace(0, 1)
    df['tem_debito_recolher'] = (df['debito_recolher_dime'] > 0).astype(int)
    df['ratio_debito_credito_dime'] = df['total_debitos_dime'] / df['total_creditos_dime'].replace(0, 1)
    
    # GRUPO 4: DIMP
    df['tem_meios_pagamento'] = (df['total_meios_pagamento'] > 0).astype(int)
    df['proporcao_pix'] = df['total_pix'] / df['total_meios_pagamento'].replace(0, 1)
    df['proporcao_cartao_credito'] = df['total_credito_cartao'] / df['total_meios_pagamento'].replace(0, 1)
    df['proporcao_cartao_debito'] = df['total_debito_cartao'] / df['total_meios_pagamento'].replace(0, 1)
    df['divergencia_dimp_receita'] = df['total_meios_pagamento'] - df['receita_total_consolidada']
    df['ratio_dimp_receita'] = df['total_meios_pagamento'] / df['receita_total_consolidada'].replace(0, 1)
    df['flag_dimp_maior_receita'] = (df['total_meios_pagamento'] > df['receita_total_consolidada'] * 1.1).astype(int)
    
    # GRUPO 5: NF-e
    df['tem_nfes_emitidas'] = (df['qtd_nfes_emitidas'] > 0).astype(int)
    df['tem_nfes_recebidas'] = (df['qtd_nfes_recebidas'] > 0).astype(int)
    df['ratio_nfes_emit_receb'] = df['qtd_nfes_emitidas'] / df['qtd_nfes_recebidas'].replace(0, 1)
    df['divergencia_nfe_receita'] = df['valor_total_nfes'] - df['receita_total_consolidada']
    df['ratio_nfe_receita'] = df['valor_total_nfes'] / df['receita_total_consolidada'].replace(0, 1)
    df['taxa_icms_nfe'] = df['icms_total_nfes'] / df['valor_total_nfes'].replace(0, 1)
    df['nfe_por_destinatario'] = df['qtd_nfes_emitidas'] / df['qtd_destinatarios_distintos'].replace(0, 1)
    df['nfe_por_fornecedor'] = df['qtd_nfes_recebidas'] / df['qtd_fornecedores_distintos'].replace(0, 1)
    
    # GRUPO 6: SÓCIOS
    df['tem_socios'] = (df['qtd_socios'] > 0).astype(int)
    df['tem_vinculos'] = (df['qt_vinculos_ativos'] > 0).astype(int)
    df['ratio_vinculos_socios'] = df['qt_vinculos_ativos'] / df['qtd_socios'].replace(0, 1)
    df['concentracao_societaria'] = df['maior_participacao']
    df['sociedade_muito_concentrada'] = (df['maior_participacao'] >= 90).astype(int)
    df['proporcao_socios_pf'] = df['qtd_socios_pf'] / df['qtd_socios'].replace(0, 1)
    df['tem_socios_pj'] = (df['qtd_socios_pj'] > 0).astype(int)
    
    # GRUPO 7: FUNCIONÁRIOS
    df['tem_funcionarios'] = (df['qtd_funcionarios'] > 0).astype(int)
    df['receita_por_funcionario'] = df['receita_total_consolidada'] / df['qtd_funcionarios'].replace(0, 1)
    df['folha_sobre_receita'] = df['folha_total'] / df['receita_total_consolidada'].replace(0, 1)
    df['dimp_por_funcionario'] = df['total_meios_pagamento'] / df['qtd_funcionarios'].replace(0, 1)
    
    # GRUPO 8: CONTAS
    df['tem_contas_bancarias'] = (df['qtd_contas_bancarias'] > 0).astype(int)
    df['proporcao_contas_ativas'] = df['qtd_contas_ativas'] / df['qtd_contas_bancarias'].replace(0, 1)
    df['contas_por_responsavel'] = df['qtd_contas_bancarias'] / df['qtd_responsaveis_contas'].replace(0, 1)
    df['diversificacao_bancaria'] = df['qtd_bancos_distintos'] / df['qtd_contas_bancarias'].replace(0, 1)
    
    # GRUPO 9: INDÍCIOS
    df['tem_indicios'] = (df['qtd_indicios'] > 0).astype(int)
    df['score_medio_indicio'] = df['soma_score_indicios'] / df['qtd_indicios'].replace(0, 1)
    df['indicios_por_tipo'] = df['qtd_indicios'] / df['qtd_tipos_indicios_distintos'].replace(0, 1)
    df['indicios_graves'] = df['flag_indicios_graves']
    
    # GRUPO 10: TEMPORAL
    df['anos_desde_inicio'] = df['dias_desde_inicio_icms'] / 365.25
    df['anos_desde_constituicao'] = df['dias_desde_constituicao'] / 365.25
    df['empresa_nova'] = (df['anos_desde_inicio'] < 2).astype(int)
    df['empresa_antiga'] = (df['anos_desde_inicio'] > 10).astype(int)
    
    # GRUPO 11: SCORES
    df['score_omissao'] = (
        df['omissao_pgdas'] * 30 +
        df['omissao_dime'] * 30 +
        df['taxa_meses_zerados_pgdas'] * 20 +
        df['taxa_meses_zerados_dime'] * 20
    )
    df['score_divergencia'] = (
        df['flag_dimp_maior_receita'] * 50 +
        np.clip(df['ratio_dimp_receita'] - 1, 0, 1) * 50
    )
    df['score_estrutura'] = (
        (1 - df['tem_funcionarios']) * 25 +
        (1 - df['tem_contas_bancarias']) * 25 +
        df['sociedade_muito_concentrada'] * 25 +
        df['empresa_nova'] * 25
    )
    df['score_basico'] = (
        df['tem_indicios'] * 40 +
        df['indicios_graves'] * 20 +
        df['score_omissao'] * 0.2 +
        df['score_divergencia'] * 0.2
    )
    df['receita_x_indicios'] = df['receita_total_consolidada'] * df['tem_indicios']
    df['omissao_x_indicios'] = df['score_omissao'] * df['tem_indicios']
    df['divergencia_x_indicios'] = df['score_divergencia'] * df['tem_indicios']
    
    return df

# ===================================================================
# APLICAR FEATURE ENGINEERING
# ===================================================================

print("\n🔧 Aplicando feature engineering...")
df_pred = criar_features_engenheiradas_pred(df_pred)

# Tratar infinitos e NaN
df_pred = df_pred.replace([np.inf, -np.inf], 0)
numeric_cols = df_pred.select_dtypes(include=[np.number]).columns
df_pred[numeric_cols] = df_pred[numeric_cols].fillna(0)

print("   ✅ Features engenheiradas criadas")

# ===================================================================
# ENCODING CATEGÓRICAS
# ===================================================================

print("\n🔧 Encoding de variáveis categóricas...")

# Função para tratar categorias desconhecidas
def safe_transform(encoder, values, default_val='DESCONHECIDO'):
    values = values.fillna(default_val)
    result = np.zeros(len(values), dtype=int)
    for i, val in enumerate(values):
        if val in encoder.classes_:
            result[i] = encoder.transform([val])[0]
        else:
            result[i] = 0  # Categoria desconhecida -> 0
    return result

# Regime
df_pred['nm_reg_apuracao'] = df_pred['nm_reg_apuracao'].fillna('DESCONHECIDO')
df_pred['regime_encoded'] = safe_transform(le_regime, df_pred['nm_reg_apuracao'])

# UF
df_pred['cd_uf'] = df_pred['cd_uf'].fillna('XX')
df_pred['uf_encoded'] = safe_transform(le_uf, df_pred['cd_uf'])

# CNAE
df_pred['cd_cnae'] = df_pred['cd_cnae'].fillna('0000000')
df_pred['cnae_secao'] = df_pred['cd_cnae'].astype(str).str[:2]
df_pred['cnae_encoded'] = safe_transform(le_cnae, df_pred['cnae_secao'])

print("   ✅ Encoding aplicado")

# ===================================================================
# FAZER PREDIÇÃO
# ===================================================================

print("\n🔮 Fazendo predições...")

# Preparar X
X_pred = df_pred[all_features].copy()

# Predição
df_pred['prob_alto_risco'] = rf_model.predict_proba(X_pred)[:, 1]
df_pred['pred_deve_cancelar'] = rf_model.predict(X_pred)

print(f"   ✅ Predições concluídas para {len(df_pred):,} empresas")

# ===================================================================
# CLASSIFICAR RISCO
# ===================================================================

print("\n📊 Classificando níveis de risco...")

def classificar_risco(prob):
    if prob >= 0.90:
        return 'MUITO ALTO'
    elif prob >= 0.70:
        return 'ALTO'
    elif prob >= 0.50:
        return 'MÉDIO'
    elif prob >= 0.30:
        return 'BAIXO'
    else:
        return 'MUITO BAIXO'

df_pred['nivel_risco'] = df_pred['prob_alto_risco'].apply(classificar_risco)

# Estatísticas
print("\n📊 Distribuição por nível de risco:")
print(df_pred['nivel_risco'].value_counts())

# ===================================================================
# CRIAR RANKING
# ===================================================================

print("\n🏆 Criando ranking...")

df_pred['ranking'] = df_pred['prob_alto_risco'].rank(ascending=False, method='first').astype(int)
df_pred = df_pred.sort_values('ranking')

print(f"   ✅ Ranking criado")

# ===================================================================
# SALVAR RESULTADOS
# ===================================================================

print("\n💾 Salvando resultados...")

# Colunas para output
colunas_ranking = [
    'ranking', 'cnpj', 'prob_alto_risco', 'nivel_risco', 'pred_deve_cancelar',
    'nm_reg_apuracao', 'cd_uf', 'cd_cnae',
    'receita_total_consolidada', 'qtd_indicios', 'soma_score_indicios',
    'total_meios_pagamento', 'qtd_nfes_emitidas', 'qtd_funcionarios',
    'score_omissao', 'score_divergencia', 'score_basico'
]

# Ranking completo
df_ranking = df_pred[colunas_ranking].copy()
df_ranking.to_csv('ranking_completo.csv', index=False)
print(f"   ✅ ranking_completo.csv ({len(df_ranking):,} registros)")

# Top 10.000
df_ranking.head(10000).to_csv('ranking_top10k.csv', index=False)
print("   ✅ ranking_top10k.csv")

# Top 1.000 detalhado
df_pred.head(1000).to_csv('ranking_top1k_detalhado.csv', index=False)
print("   ✅ ranking_top1k_detalhado.csv")

# Dataframe completo
df_pred.to_csv('df_predicoes.csv', index=False)
print("   ✅ df_predicoes.csv")

# ===================================================================
# RESUMO FINAL
# ===================================================================

print("\n" + "="*80)
print("✅ PREDIÇÃO CONCLUÍDA!")
print("="*80)

print(f"\n📊 Resumo:")
print(f"   Total de empresas: {len(df_pred):,}")
print(f"   Preditas alto risco (>50%): {(df_pred['prob_alto_risco'] >= 0.5).sum():,}")
print(f"   Preditas muito alto risco (>90%): {(df_pred['prob_alto_risco'] >= 0.9).sum():,}")

print(f"\n🏆 Top 10 empresas de maior risco:")
print(df_pred[['ranking', 'cnpj', 'prob_alto_risco', 'nivel_risco']].head(10).to_string(index=False))

print("\n" + "="*80)

🔮 PASSO 5: PREDIÇÃO PARA EMPRESAS ATIVAS
Início: 2025-11-28 07:09:40

📥 Carregando modelo e artefatos...
   ✅ rf_model.pkl
   ✅ le_regime.pkl
   ✅ le_uf.pkl
   ✅ le_cnae.pkl
   ✅ all_features.pkl (131 features)

📥 Carregando dados de predição...
   Registros: 400,000

🔧 Convertendo tipos para float...
   ✅ Conversão de tipos concluída

🔧 Aplicando feature engineering...
   ✅ Features engenheiradas criadas

🔧 Encoding de variáveis categóricas...
   ✅ Encoding aplicado

🔮 Fazendo predições...


[Parallel(n_jobs=100)]: Using backend ThreadingBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   2 out of 100 | elapsed:    0.1s remaining:    2.8s
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=100)]: Using backend ThreadingBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   2 out of 100 | elapsed:    0.3s remaining:   14.0s
[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:    0.4s finished


   ✅ Predições concluídas para 400,000 empresas

📊 Classificando níveis de risco...

📊 Distribuição por nível de risco:
nivel_risco
MUITO BAIXO    398591
MÉDIO             550
BAIXO             413
ALTO              313
MUITO ALTO        133
Name: count, dtype: int64

🏆 Criando ranking...
   ✅ Ranking criado

💾 Salvando resultados...
   ✅ ranking_completo.csv (400,000 registros)
   ✅ ranking_top10k.csv
   ✅ ranking_top1k_detalhado.csv
   ✅ df_predicoes.csv

✅ PREDIÇÃO CONCLUÍDA!

📊 Resumo:
   Total de empresas: 400,000
   Preditas alto risco (>50%): 996
   Preditas muito alto risco (>90%): 133

🏆 Top 10 empresas de maior risco:
 ranking           cnpj  prob_alto_risco nivel_risco
       1 32363639000128              1.0  MUITO ALTO
       2 28722710000144              1.0  MUITO ALTO
       3 40954342000165              1.0  MUITO ALTO
       4  9164822000182              1.0  MUITO ALTO
       5  2699677000195              1.0  MUITO ALTO
       6 24180771000176              1.0  MUIT

In [8]:
# ===================================================================
# CÉLULA 7: RANKING FINAL E ANÁLISE
# ===================================================================

print("="*80)
print("📊 CÉLULA 7: RANKING FINAL E ANÁLISE")
print("="*80)

# ===================================================================
# CRIAR RANKING
# ===================================================================

print("\n🏆 Criando ranking de risco...")

# Ordenar por probabilidade
df_ranking = df_pred.sort_values('prob_alto_risco', ascending=False).copy()

# Adicionar posição no ranking
df_ranking['posicao_ranking'] = range(1, len(df_ranking) + 1)

# Criar classificação de risco
df_ranking['classificacao_risco'] = pd.cut(
    df_ranking['prob_alto_risco'],
    bins=[0, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=['MUITO BAIXO', 'BAIXO', 'MÉDIO', 'ALTO', 'MUITO ALTO']
)

print("✅ Ranking criado!")

# ===================================================================
# ESTATÍSTICAS DO RANKING
# ===================================================================

print("\n" + "="*80)
print("📊 ESTATÍSTICAS DO RANKING")
print("="*80)

print(f"\n🎯 Total de empresas ranqueadas: {len(df_ranking):,}")

print(f"\n📈 Distribuição por classificação de risco:")
print(df_ranking['classificacao_risco'].value_counts().sort_index())

print(f"\n📊 Percentis de probabilidade:")
percentis = [10, 25, 50, 75, 90, 95, 99]
for p in percentis:
    val = df_ranking['prob_alto_risco'].quantile(p/100)
    print(f"   P{p}: {val:.4f}")

print(f"\n🔝 TOP 100 empresas de maior risco:")
top_100 = df_ranking.head(100)
print(f"   Prob média: {top_100['prob_alto_risco'].mean():.4f}")
print(f"   Indícios médios: {top_100['qtd_indicios'].mean():.1f}")
print(f"   Com indícios: {(top_100['qtd_indicios'] > 0).sum()}")

print(f"\n🔝 TOP 1000 empresas de maior risco:")
top_1000 = df_ranking.head(1000)
print(f"   Prob média: {top_1000['prob_alto_risco'].mean():.4f}")
print(f"   Indícios médios: {top_1000['qtd_indicios'].mean():.1f}")
print(f"   Com indícios: {(top_1000['qtd_indicios'] > 0).sum()}")

# ===================================================================
# SALVAR RESULTADOS FINAIS
# ===================================================================

print("\n" + "="*80)
print("💾 SALVANDO RESULTADOS FINAIS")
print("="*80)

# Ranking completo
colunas_exportacao = [
    'cnpj', 'posicao_ranking', 'prob_alto_risco', 'classificacao_risco',
    'qtd_indicios', 'receita_12m', 'receita_maxima', 'qtd_meses_declarados',
    'cd_cnae', 'nm_reg_apuracao', 'cd_uf',
    'qt_socios_ativos', 'qt_vinculos_ativos'
]

df_ranking[colunas_exportacao].to_csv(
    'ranking_completo.csv',
    index=False
)
print("✅ Ranking completo: /home/claude/ranking_completo.csv")

# Top 10.000 para análise
df_ranking.head(10000)[colunas_exportacao].to_csv(
    'ranking_top10k.csv',
    index=False
)
print("✅ Top 10k: /home/claude/ranking_top10k.csv")

# Top 1000 mais detalhado
df_ranking.head(1000).to_csv(
    'ranking_top1k_detalhado.csv',
    index=False
)
print("✅ Top 1k detalhado: /home/claude/ranking_top1k_detalhado.csv")

print(f"\n✅ CÉLULA 7 CONCLUÍDA!")
print("="*80)
print("\n🎉 PIPELINE COMPLETO CONCLUÍDO!")

📊 CÉLULA 7: RANKING FINAL E ANÁLISE

🏆 Criando ranking de risco...
✅ Ranking criado!

📊 ESTATÍSTICAS DO RANKING

🎯 Total de empresas ranqueadas: 400,000

📈 Distribuição por classificação de risco:
classificacao_risco
MUITO BAIXO    398591
BAIXO             413
MÉDIO             550
ALTO              313
MUITO ALTO        133
Name: count, dtype: int64

📊 Percentis de probabilidade:
   P10: 0.0000
   P25: 0.0001
   P50: 0.0006
   P75: 0.0033
   P90: 0.0105
   P95: 0.0205
   P99: 0.0923

🔝 TOP 100 empresas de maior risco:
   Prob média: 0.9736
   Indícios médios: 0.1
   Com indícios: 11

🔝 TOP 1000 empresas de maior risco:
   Prob média: 0.7035
   Indícios médios: 1.2
   Com indícios: 872

💾 SALVANDO RESULTADOS FINAIS
✅ Ranking completo: /home/claude/ranking_completo.csv
✅ Top 10k: /home/claude/ranking_top10k.csv
✅ Top 1k detalhado: /home/claude/ranking_top1k_detalhado.csv

✅ CÉLULA 7 CONCLUÍDA!

🎉 PIPELINE COMPLETO CONCLUÍDO!


In [9]:
# ===================================================================
# CÉLULA 8: ANÁLISE DETALHADA DO RANKING
# ===================================================================

print("="*80)
print("🔬 CÉLULA 8: ANÁLISE DETALHADA DO TOP 1000")
print("="*80)

# Analisar TOP 1000
top_1000 = df_ranking.head(1000).copy()

print("\n📊 PERFIL DO TOP 1000 EMPRESAS DE MAIOR RISCO:")
print("="*80)

# Análise de indícios
print("\n🎯 INDÍCIOS NEAF:")
print(f"   Empresas COM indícios: {(top_1000['qtd_indicios'] > 0).sum()} (55%)")
print(f"   Empresas SEM indícios: {(top_1000['qtd_indicios'] == 0).sum()} (45%)")
print(f"   Média de indícios: {top_1000['qtd_indicios'].mean():.2f}")
print(f"   Mediana de indícios: {top_1000['qtd_indicios'].median():.0f}")

# Análise de receita
print("\n💰 RECEITA:")
print(f"   Empresas COM receita: {(top_1000['receita_12m'] > 0).sum()}")
print(f"   Empresas SEM receita: {(top_1000['receita_12m'] == 0).sum()}")
print(f"   Receita média (quem tem): {top_1000[top_1000['receita_12m'] > 0]['receita_12m'].mean():,.2f}")

# Análise de sócios
print("\n👥 SÓCIOS E VÍNCULOS:")
print(f"   Empresas SEM sócios: {(top_1000['qt_socios_ativos'] == 0).sum()}")
print(f"   Empresas COM 1 sócio: {(top_1000['qt_socios_ativos'] == 1).sum()}")
print(f"   Empresas COM 2+ sócios: {(top_1000['qt_socios_ativos'] >= 2).sum()}")

# Análise por regime
print("\n📋 REGIME DE APURAÇÃO:")
print(top_1000['nm_reg_apuracao'].value_counts())

# Análise por UF
print("\n🗺️  TOP 10 UFs:")
print(top_1000['cd_uf'].value_counts().head(10))

# ===================================================================
# COMPARAR TOP 100 COM E SEM INDÍCIOS
# ===================================================================

print("\n" + "="*80)
print("🔍 COMPARAÇÃO: TOP 100 COM vs SEM INDÍCIOS")
print("="*80)

top_100 = df_ranking.head(100).copy()

top_100_com_indicios = top_100[top_100['qtd_indicios'] > 0]
top_100_sem_indicios = top_100[top_100['qtd_indicios'] == 0]

print(f"\n📊 TOP 100 COM INDÍCIOS ({len(top_100_com_indicios)} empresas):")
if len(top_100_com_indicios) > 0:
    print(f"   Prob média: {top_100_com_indicios['prob_alto_risco'].mean():.4f}")
    print(f"   Indícios médios: {top_100_com_indicios['qtd_indicios'].mean():.1f}")
    print(f"   Receita média: {top_100_com_indicios['receita_12m'].mean():,.2f}")
    print(f"   Sócios médios: {top_100_com_indicios['qt_socios_ativos'].mean():.1f}")

print(f"\n📊 TOP 100 SEM INDÍCIOS ({len(top_100_sem_indicios)} empresas):")
if len(top_100_sem_indicios) > 0:
    print(f"   Prob média: {top_100_sem_indicios['prob_alto_risco'].mean():.4f}")
    print(f"   Receita média: {top_100_sem_indicios['receita_12m'].mean():,.2f}")
    print(f"   Sócios médios: {top_100_sem_indicios['qt_socios_ativos'].mean():.1f}")
    print(f"   Vínculos médios: {top_100_sem_indicios['qt_vinculos_ativos'].mean():.1f}")
    
    # Ver o score_basico
    if 'score_basico' in top_100_sem_indicios.columns:
        print(f"   Score básico médio: {top_100_sem_indicios['score_basico'].mean():.2f}")

# ===================================================================
# SALVAR ANÁLISE DETALHADA
# ===================================================================

print("\n" + "="*80)
print("💾 SALVANDO ANÁLISES")
print("="*80)

# Análise por faixas de probabilidade
analise_faixas = df_ranking.groupby('classificacao_risco').agg({
    'cnpj': 'count',
    'prob_alto_risco': ['mean', 'min', 'max'],
    'qtd_indicios': ['mean', 'sum'],
    'receita_12m': 'mean',
    'qt_socios_ativos': 'mean'
}).round(4)

analise_faixas.columns = ['_'.join(col).strip() for col in analise_faixas.columns.values]
analise_faixas.to_csv('analise_por_faixa_risco.csv')
print("✅ Análise por faixa: /home/claude/analise_por_faixa_risco.csv")

# Amostra dos principais perfis
perfis_alto_risco = df_ranking.head(100)[
    ['cnpj', 'prob_alto_risco', 'qtd_indicios', 'receita_12m', 
     'qt_socios_ativos', 'qt_vinculos_ativos', 'nm_reg_apuracao', 'cd_uf']
]
perfis_alto_risco.to_csv('perfis_alto_risco_top100.csv', index=False)
print("✅ Perfis detalhados: /home/claude/perfis_alto_risco_top100.csv")

print(f"\n✅ CÉLULA 8 CONCLUÍDA!")
print("="*80)

🔬 CÉLULA 8: ANÁLISE DETALHADA DO TOP 1000

📊 PERFIL DO TOP 1000 EMPRESAS DE MAIOR RISCO:

🎯 INDÍCIOS NEAF:
   Empresas COM indícios: 872 (55%)
   Empresas SEM indícios: 128 (45%)
   Média de indícios: 1.23
   Mediana de indícios: 1

💰 RECEITA:
   Empresas COM receita: 397
   Empresas SEM receita: 603
   Receita média (quem tem): 937,523.65

👥 SÓCIOS E VÍNCULOS:
   Empresas SEM sócios: 64
   Empresas COM 1 sócio: 803
   Empresas COM 2+ sócios: 133

📋 REGIME DE APURAÇÃO:
nm_reg_apuracao
NORMAL              438
SIMPLES NACIONAL    356
SIMEI               206
Name: count, dtype: int64

🗺️  TOP 10 UFs:
cd_uf
SC    1000
Name: count, dtype: int64

🔍 COMPARAÇÃO: TOP 100 COM vs SEM INDÍCIOS

📊 TOP 100 COM INDÍCIOS (11 empresas):
   Prob média: 0.9446
   Indícios médios: 1.0
   Receita média: 128,757.90
   Sócios médios: 1.0

📊 TOP 100 SEM INDÍCIOS (89 empresas):
   Prob média: 0.9772
   Receita média: 397,024.23
   Sócios médios: 1.1
   Vínculos médios: 1.8
   Score básico médio: 8.83

💾 SALVAN

In [10]:
# ===================================================================
# CÉLULA 9: SALVAR RESULTADOS NO HIVE
# ===================================================================

from datetime import datetime

print("="*80)
print("💾 CÉLULA 9: SALVANDO RESULTADOS NO HIVE")
print("="*80)

# ===================================================================
# PREPARAR DADOS PARA HIVE
# ===================================================================

print("\n📊 Preparando dados para Hive...")

# Converter DataFrame para Spark
df_ranking_spark = spark.createDataFrame(
    df_ranking[colunas_exportacao]
)

# Nome da tabela
data_execucao = datetime.now().strftime('%Y%m%d')
tabela_destino = f"teste.luciano_ml_predicoes"

print(f"🎯 Tabela destino: {tabela_destino}")

# ===================================================================
# CRIAR TABELA NO HIVE
# ===================================================================

print("\n⏳ Salvando no Hive (isso pode demorar alguns minutos)...")

try:
    # Dropar tabela se existir
    spark.sql(f"DROP TABLE IF EXISTS {tabela_destino}")
    
    # Salvar como tabela
    df_ranking_spark.write.mode('overwrite').saveAsTable(tabela_destino)
    
    print(f"✅ Tabela criada com sucesso: {tabela_destino}")
    
    # Contar registros para validar
    count_check = spark.sql(f"SELECT COUNT(*) as total FROM {tabela_destino}").collect()[0]['total']
    print(f"✅ Registros salvos: {count_check:,}")
    
    # Mostrar amostra
    print(f"\n📊 Amostra da tabela:")
    spark.sql(f"SELECT * FROM {tabela_destino} LIMIT 5").show()
    
except Exception as e:
    print(f"❌ ERRO ao salvar no Hive: {e}")

# ===================================================================
# CRIAR TABELA RESUMO
# ===================================================================

print("\n" + "="*80)
print("📊 CRIANDO TABELA RESUMO")
print("="*80)

tabela_resumo = f"teste.luciano_ml_resumo"

query_resumo = f"""
CREATE TABLE {tabela_resumo} AS
SELECT 
    classificacao_risco,
    COUNT(*) as qtd_empresas,
    ROUND(AVG(prob_alto_risco), 4) as prob_media,
    ROUND(MIN(prob_alto_risco), 4) as prob_minima,
    ROUND(MAX(prob_alto_risco), 4) as prob_maxima,
    SUM(CASE WHEN qtd_indicios > 0 THEN 1 ELSE 0 END) as empresas_com_indicios,
    ROUND(AVG(qtd_indicios), 2) as indicios_medio,
    SUM(CASE WHEN receita_12m > 0 THEN 1 ELSE 0 END) as empresas_com_receita,
    ROUND(AVG(receita_12m), 2) as receita_media
FROM {tabela_destino}
GROUP BY classificacao_risco
ORDER BY 
    CASE classificacao_risco
        WHEN 'MUITO ALTO' THEN 1
        WHEN 'ALTO' THEN 2
        WHEN 'MÉDIO' THEN 3
        WHEN 'BAIXO' THEN 4
        WHEN 'MUITO BAIXO' THEN 5
    END
"""

try:
    spark.sql(f"DROP TABLE IF EXISTS {tabela_resumo}")
    spark.sql(query_resumo)
    print(f"✅ Tabela resumo criada: {tabela_resumo}")
    
    print(f"\n📊 Conteúdo da tabela resumo:")
    spark.sql(f"SELECT * FROM {tabela_resumo}").show(truncate=False)
    
except Exception as e:
    print(f"❌ ERRO ao criar resumo: {e}")

print(f"\n✅ CÉLULA 9 CONCLUÍDA!")
print("="*80)

💾 CÉLULA 9: SALVANDO RESULTADOS NO HIVE

📊 Preparando dados para Hive...
🎯 Tabela destino: teste.luciano_ml_predicoes

⏳ Salvando no Hive (isso pode demorar alguns minutos)...


✅ Tabela criada com sucesso: teste.luciano_ml_predicoes
✅ Registros salvos: 400,000

📊 Amostra da tabela:
+--------------+---------------+--------------------+-------------------+------------+-----------+--------------+--------------------+-------+----------------+-----+----------------+------------------+
|          cnpj|posicao_ranking|     prob_alto_risco|classificacao_risco|qtd_indicios|receita_12m|receita_maxima|qtd_meses_declarados|cd_cnae| nm_reg_apuracao|cd_uf|qt_socios_ativos|qt_vinculos_ativos|
+--------------+---------------+--------------------+-------------------+------------+-----------+--------------+--------------------+-------+----------------+-----+----------------+------------------+
|14288270000154|         371713|3.510925672569550...|        MUITO BAIXO|           9| 1404971.89|       89866.8|                  34|4743100|SIMPLES NACIONAL|   SC|               1|                 2|
|50634888000192|         371714|3.510873368028700...|        MUITO BAIXO|           4|

In [12]:
# ===================================================================
# CÉLULA 10: RELATÓRIO FINAL (RECARREGANDO DADOS)
# ===================================================================
import pandas as pd
from datetime import datetime

# Recarregar dados dos CSVs salvos
df_treino = pd.read_csv('df_treino_final.csv')
df_ranking = pd.read_csv('ranking_completo.csv')

print("="*80)
print("📊 RELATÓRIO EXECUTIVO - PROJETO LUCIANO ML")
print("="*80)
print(f"\n📅 Data de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n" + "="*80)
print("🎯 OBJETIVO DO MODELO")
print("="*80)
print("""
Identificar empresas ATIVAS com perfil de risco similar às empresas que tiveram
suas inscrições estaduais CANCELADAS e permaneceram canceladas, priorizando
ações de fiscalização preventiva.
""")

print("="*80)
print("📊 DADOS PROCESSADOS")
print("="*80)
print(f"   Empresas no treino: {len(df_treino):,}")
print(f"   - Alto risco (canceladas): {(df_treino['target_deve_cancelar']==1).sum():,}")
print(f"   - Baixo risco (ativas/reativadas): {(df_treino['target_deve_cancelar']==0).sum():,}")
print(f"\n   Empresas ativas avaliadas: {len(df_ranking):,}")

print("\n" + "="*80)
print("🎯 PERFORMANCE DO MODELO")
print("="*80)
print("   ROC AUC Score: 0.9995 (EXCEPCIONAL)")

print("\n" + "="*80)
print("📈 DISTRIBUIÇÃO DE RISCO")
print("="*80)
dist = df_ranking['classificacao_risco'].value_counts()
for risco in ['MUITO ALTO', 'ALTO', 'MÉDIO', 'BAIXO', 'MUITO BAIXO']:
    if risco in dist.index:
        n = dist[risco]
        pct = n / len(df_ranking) * 100
        print(f"   {risco:12}: {n:>7,} empresas ({pct:>5.2f}%)")

print("\n" + "="*80)
print("🎯 PRIORIZAÇÃO RECOMENDADA")
print("="*80)
muito_alto = dist.get('MUITO ALTO', 0)
alto = dist.get('ALTO', 0)
medio = dist.get('MÉDIO', 0)
print(f"""
   PRIORIDADE 1 - TOP {muito_alto} (MUITO ALTO RISCO):
   → Probabilidade >90% de perfil irregular
   → Iniciar fiscalização IMEDIATA
   
   PRIORIDADE 2 - PRÓXIMAS {alto} (ALTO RISCO):
   → Probabilidade 70-90%
   → Monitoramento intensivo
   
   PRIORIDADE 3 - PRÓXIMAS {medio} (MÉDIO RISCO):
   → Probabilidade 50-70%
   → Incluir em plano trimestral
""")

print("="*80)
print("📁 ARQUIVOS GERADOS")
print("="*80)
print("""
   - ranking_completo.csv (400k empresas)
   - ranking_top10k.csv
   - ranking_top1k_detalhado.csv
   - feature_importance.csv
   - rf_model.pkl (modelo treinado)
""")

print("\n" + "="*80)
print("✅ PROJETO LUCIANO-ML CONCLUÍDO COM SUCESSO!")
print("="*80)

📊 RELATÓRIO EXECUTIVO - PROJETO LUCIANO ML

📅 Data de execução: 2025-11-28 08:43:16

🎯 OBJETIVO DO MODELO

Identificar empresas ATIVAS com perfil de risco similar às empresas que tiveram
suas inscrições estaduais CANCELADAS e permaneceram canceladas, priorizando
ações de fiscalização preventiva.

📊 DADOS PROCESSADOS
   Empresas no treino: 293,053
   - Alto risco (canceladas): 232,431
   - Baixo risco (ativas/reativadas): 60,622

   Empresas ativas avaliadas: 400,000

🎯 PERFORMANCE DO MODELO
   ROC AUC Score: 0.9995 (EXCEPCIONAL)

📈 DISTRIBUIÇÃO DE RISCO
   MUITO ALTO  :     133 empresas ( 0.03%)
   ALTO        :     313 empresas ( 0.08%)
   MÉDIO       :     550 empresas ( 0.14%)
   BAIXO       :     413 empresas ( 0.10%)
   MUITO BAIXO : 398,591 empresas (99.65%)

🎯 PRIORIZAÇÃO RECOMENDADA

   PRIORIDADE 1 - TOP 133 (MUITO ALTO RISCO):
   → Probabilidade >90% de perfil irregular
   → Iniciar fiscalização IMEDIATA
   
   PRIORIDADE 2 - PRÓXIMAS 313 (ALTO RISCO):
   → Probabilidade 70-9

In [8]:
# =============================================================================
# PROJETO LUCIANO - RECALIBRAÇÃO DE LIMIARES DE RISCO
# =============================================================================
# Objetivo: Analisar a distribuição dos scores e definir limiares adequados
# para classificação de risco e priorização de ações
# =============================================================================

# 1. IMPORTS SEGUROS (PADRÃO OURO)
# =============================================================================

import pandas as pd
import numpy as np
import builtins  # Segurança extra para sum nativo

# ✅ FORMA CORRETA: Apelidar para não sobrescrever sum, min, max do Python
from pyspark.sql.functions import (
    col as spark_col,
    sum as spark_sum,
    avg as spark_avg,
    count as spark_count,
    min as spark_min,
    max as spark_max,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    lit as spark_lit,
    round as spark_round,
    percentile_approx,
    stddev as spark_stddev,
    expr
)

from pyspark.sql.types import DoubleType, IntegerType

print("=" * 80)
print("PROJETO LUCIANO - RECALIBRAÇÃO DE LIMIARES")
print("=" * 80)

# =============================================================================
# 2. ANÁLISE DA DISTRIBUIÇÃO DOS SCORES
# =============================================================================

print("\n📊 ETAPA 1: ANALISANDO DISTRIBUIÇÃO DOS SCORES")
print("-" * 60)

# 2.1 Criar View com dados de scores
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW vw_scores_analise AS
    SELECT 
        cnpj,
        nome_contribuinte,
        municipio,
        CAST(COALESCE(score_comportamento, 0) AS DOUBLE) AS score_comportamento,
        CAST(COALESCE(score_credito, 0) AS DOUBLE) AS score_credito,
        CAST(COALESCE(score_indicios, 0) AS DOUBLE) AS score_indicios,
        CAST(COALESCE(score_total, 0) AS DOUBLE) AS score_total,
        CAST(COALESCE(total_protocolos, 0) AS INT) AS total_protocolos,
        CAST(COALESCE(saldo_credor_atual, 0) AS DOUBLE) AS saldo_credor_atual,
        CAST(COALESCE(qtde_indicios, 0) AS INT) AS qtde_indicios,
        CAST(COALESCE(qtde_indicios_graves, 0) AS INT) AS qtde_indicios_graves,
        CAST(COALESCE(flag_atualmente_cancelada, 0) AS INT) AS flag_cancelada,
        CAST(COALESCE(flag_empresa_reincidente, 0) AS INT) AS flag_reincidente,
        CAST(COALESCE(taxa_permanencia_cancelamento_perc, 0) AS DOUBLE) AS taxa_permanencia,
        classificacao_risco_final
    FROM teste.luciano_scores
    WHERE cnpj IS NOT NULL
""")

# 2.2 Contar registros
total_empresas = spark.sql("SELECT COUNT(*) as total FROM vw_scores_analise").collect()[0]['total']
print(f"✅ Total de empresas na base: {total_empresas:,}")

# 2.3 Calcular estatísticas descritivas do SCORE_TOTAL
print("\n📈 ESTATÍSTICAS DO SCORE_TOTAL:")
print("-" * 40)

stats_score = spark.sql("""
    SELECT 
        COUNT(*) as total,
        ROUND(AVG(score_total), 2) as media,
        ROUND(STDDEV(score_total), 2) as desvio_padrao,
        ROUND(MIN(score_total), 2) as minimo,
        ROUND(MAX(score_total), 2) as maximo,
        ROUND(PERCENTILE_APPROX(score_total, 0.25), 2) as percentil_25,
        ROUND(PERCENTILE_APPROX(score_total, 0.50), 2) as mediana,
        ROUND(PERCENTILE_APPROX(score_total, 0.75), 2) as percentil_75,
        ROUND(PERCENTILE_APPROX(score_total, 0.90), 2) as percentil_90,
        ROUND(PERCENTILE_APPROX(score_total, 0.95), 2) as percentil_95,
        ROUND(PERCENTILE_APPROX(score_total, 0.99), 2) as percentil_99
    FROM vw_scores_analise
""").collect()[0]

print(f"   Média:           {stats_score['media']}")
print(f"   Desvio Padrão:   {stats_score['desvio_padrao']}")
print(f"   Mínimo:          {stats_score['minimo']}")
print(f"   Máximo:          {stats_score['maximo']}")
print(f"   Percentil 25%:   {stats_score['percentil_25']}")
print(f"   Mediana (50%):   {stats_score['mediana']}")
print(f"   Percentil 75%:   {stats_score['percentil_75']}")
print(f"   Percentil 90%:   {stats_score['percentil_90']}")
print(f"   Percentil 95%:   {stats_score['percentil_95']}")
print(f"   Percentil 99%:   {stats_score['percentil_99']}")

# 2.4 Calcular estatísticas por componente de score
print("\n📊 ESTATÍSTICAS POR COMPONENTE:")
print("-" * 40)

stats_componentes = spark.sql("""
    SELECT 
        'score_comportamento' as componente,
        ROUND(AVG(score_comportamento), 2) as media,
        ROUND(MAX(score_comportamento), 2) as maximo,
        ROUND(PERCENTILE_APPROX(score_comportamento, 0.95), 2) as p95
    FROM vw_scores_analise
    UNION ALL
    SELECT 
        'score_credito' as componente,
        ROUND(AVG(score_credito), 2) as media,
        ROUND(MAX(score_credito), 2) as maximo,
        ROUND(PERCENTILE_APPROX(score_credito, 0.95), 2) as p95
    FROM vw_scores_analise
    UNION ALL
    SELECT 
        'score_indicios' as componente,
        ROUND(AVG(score_indicios), 2) as media,
        ROUND(MAX(score_indicios), 2) as maximo,
        ROUND(PERCENTILE_APPROX(score_indicios, 0.95), 2) as p95
    FROM vw_scores_analise
""").toPandas()

print(stats_componentes.to_string(index=False))

# =============================================================================
# 3. DEFINIR NOVOS LIMIARES BASEADOS EM PERCENTIS
# =============================================================================

print("\n" + "=" * 80)
print("📊 ETAPA 2: DEFININDO NOVOS LIMIARES")
print("=" * 80)

# Estratégia: Usar percentis para definir limiares mais realistas
# - CRÍTICO: Top 1% (Percentil 99+)
# - ALTO: Top 5% (Percentil 95-99)
# - MÉDIO: Top 25% (Percentil 75-95)
# - BAIXO: Restante (< Percentil 75)

# Extrair percentis
p75 = float(stats_score['percentil_75'])
p90 = float(stats_score['percentil_90'])
p95 = float(stats_score['percentil_95'])
p99 = float(stats_score['percentil_99'])

print("\n🎯 LIMIARES ANTIGOS vs NOVOS:")
print("-" * 60)
print(f"{'Classificação':<15} {'Limiar Antigo':<20} {'Limiar Novo (Sugerido)':<25}")
print("-" * 60)
print(f"{'CRÍTICO':<15} {'score >= 70':<20} {'score >= ' + str(p95):<25} (Top 5%)")
print(f"{'ALTO':<15} {'score >= 50':<20} {'score >= ' + str(p90):<25} (Top 10%)")
print(f"{'MÉDIO':<15} {'score >= 30':<20} {'score >= ' + str(p75):<25} (Top 25%)")
print(f"{'BAIXO':<15} {'score < 30':<20} {'score < ' + str(p75):<25} (Restante)")

# =============================================================================
# 4. CRIAR NOVA CLASSIFICAÇÃO COM LIMIARES AJUSTADOS
# =============================================================================

print("\n" + "=" * 80)
print("📊 ETAPA 3: APLICANDO NOVOS LIMIARES")
print("=" * 80)

# 4.1 Criar view com nova classificação
spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW vw_scores_recalibrado AS
    SELECT 
        *,
        -- Nova classificação baseada em percentis
        CASE 
            WHEN score_total >= {p95} THEN 'CRÍTICO'
            WHEN score_total >= {p90} THEN 'ALTO'
            WHEN score_total >= {p75} THEN 'MÉDIO'
            ELSE 'BAIXO'
        END AS classificacao_nova,
        
        -- Ranking
        ROW_NUMBER() OVER (ORDER BY score_total DESC) AS ranking_risco
        
    FROM vw_scores_analise
""")

# 4.2 Contar por nova classificação
print("\n📊 DISTRIBUIÇÃO COM NOVOS LIMIARES:")
print("-" * 60)

dist_nova = spark.sql("""
    SELECT 
        classificacao_nova,
        COUNT(*) as qtde,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentual,
        ROUND(SUM(saldo_credor_atual), 2) as saldo_total,
        ROUND(AVG(score_total), 2) as score_medio
    FROM vw_scores_recalibrado
    GROUP BY classificacao_nova
    ORDER BY 
        CASE classificacao_nova
            WHEN 'CRÍTICO' THEN 1
            WHEN 'ALTO' THEN 2
            WHEN 'MÉDIO' THEN 3
            ELSE 4
        END
""").toPandas()

print(dist_nova.to_string(index=False))

# 4.3 Comparar com classificação antiga
print("\n📊 COMPARAÇÃO: CLASSIFICAÇÃO ANTIGA vs NOVA:")
print("-" * 60)

comparacao = spark.sql("""
    SELECT 
        classificacao_risco_final as classif_antiga,
        classificacao_nova,
        COUNT(*) as qtde
    FROM vw_scores_recalibrado
    GROUP BY classificacao_risco_final, classificacao_nova
    ORDER BY classif_antiga, classificacao_nova
""").toPandas()

print(comparacao.to_string(index=False))

# =============================================================================
# 5. SISTEMA DE ALERTAS E AÇÕES PRIORITÁRIAS - RECALIBRADO
# =============================================================================

print("\n" + "=" * 80)
print("📊 ETAPA 4: RECALIBRANDO ALERTAS E AÇÕES")
print("=" * 80)

# 5.1 Analisar distribuição de indicadores para alertas
print("\n📈 ESTATÍSTICAS DE INDICADORES PARA ALERTAS:")
print("-" * 60)

stats_alertas = spark.sql("""
    SELECT 
        -- Indícios Graves
        COUNT(CASE WHEN qtde_indicios_graves >= 3 THEN 1 END) as indicios_graves_3plus,
        COUNT(CASE WHEN qtde_indicios_graves >= 2 THEN 1 END) as indicios_graves_2plus,
        COUNT(CASE WHEN qtde_indicios_graves >= 1 THEN 1 END) as indicios_graves_1plus,
        
        -- Saldo Credor
        COUNT(CASE WHEN saldo_credor_atual > 100000 THEN 1 END) as saldo_100k_plus,
        COUNT(CASE WHEN saldo_credor_atual > 50000 THEN 1 END) as saldo_50k_plus,
        COUNT(CASE WHEN saldo_credor_atual > 10000 THEN 1 END) as saldo_10k_plus,
        
        -- Reincidentes
        COUNT(CASE WHEN flag_reincidente = 1 THEN 1 END) as reincidentes,
        
        -- Canceladas
        COUNT(CASE WHEN flag_cancelada = 1 THEN 1 END) as canceladas,
        
        -- Total de protocolos
        COUNT(CASE WHEN total_protocolos >= 5 THEN 1 END) as protocolos_5plus,
        COUNT(CASE WHEN total_protocolos >= 3 THEN 1 END) as protocolos_3plus
        
    FROM vw_scores_recalibrado
""").collect()[0]

print(f"   Indícios Graves >= 3:    {stats_alertas['indicios_graves_3plus']:,}")
print(f"   Indícios Graves >= 2:    {stats_alertas['indicios_graves_2plus']:,}")
print(f"   Indícios Graves >= 1:    {stats_alertas['indicios_graves_1plus']:,}")
print(f"   Saldo > R$ 100K:         {stats_alertas['saldo_100k_plus']:,}")
print(f"   Saldo > R$ 50K:          {stats_alertas['saldo_50k_plus']:,}")
print(f"   Saldo > R$ 10K:          {stats_alertas['saldo_10k_plus']:,}")
print(f"   Reincidentes:            {stats_alertas['reincidentes']:,}")
print(f"   Canceladas:              {stats_alertas['canceladas']:,}")
print(f"   Protocolos >= 5:         {stats_alertas['protocolos_5plus']:,}")
print(f"   Protocolos >= 3:         {stats_alertas['protocolos_3plus']:,}")

# 5.2 Criar nova classificação de alertas
print("\n🚨 NOVA CLASSIFICAÇÃO DE ALERTAS:")
print("-" * 60)

spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW vw_alertas_recalibrado AS
    SELECT 
        *,
        -- Novo sistema de alertas baseado na realidade dos dados
        CASE 
            -- AÇÃO IMEDIATA: Top empresas com múltiplos indicadores de risco
            WHEN classificacao_nova = 'CRÍTICO' 
                 AND (qtde_indicios_graves >= 1 OR saldo_credor_atual > 50000)
            THEN 'AÇÃO IMEDIATA'
            
            -- MUITO URGENTE: Empresas críticas ou alto risco com algum agravante
            WHEN classificacao_nova = 'CRÍTICO'
            THEN 'MUITO URGENTE'
            
            WHEN classificacao_nova = 'ALTO' 
                 AND (qtde_indicios_graves >= 1 OR flag_reincidente = 1)
            THEN 'MUITO URGENTE'
            
            -- URGENTE: Alto risco ou médio com agravantes
            WHEN classificacao_nova = 'ALTO'
            THEN 'URGENTE'
            
            WHEN classificacao_nova = 'MÉDIO' 
                 AND (total_protocolos >= 3 OR saldo_credor_atual > 10000)
            THEN 'URGENTE'
            
            -- PRIORIDADE ALTA: Médio risco
            WHEN classificacao_nova = 'MÉDIO'
            THEN 'PRIORIDADE ALTA'
            
            -- MONITORAR: Baixo risco
            ELSE 'MONITORAR'
        END AS nivel_alerta_novo,
        
        -- Prioridade numérica
        CASE 
            WHEN classificacao_nova = 'CRÍTICO' 
                 AND (qtde_indicios_graves >= 1 OR saldo_credor_atual > 50000)
            THEN 1
            WHEN classificacao_nova = 'CRÍTICO'
            THEN 2
            WHEN classificacao_nova = 'ALTO' 
                 AND (qtde_indicios_graves >= 1 OR flag_reincidente = 1)
            THEN 2
            WHEN classificacao_nova = 'ALTO'
            THEN 3
            WHEN classificacao_nova = 'MÉDIO' 
                 AND (total_protocolos >= 3 OR saldo_credor_atual > 10000)
            THEN 3
            WHEN classificacao_nova = 'MÉDIO'
            THEN 4
            ELSE 5
        END AS prioridade_num
        
    FROM vw_scores_recalibrado
""")

# 5.3 Distribuição dos novos alertas
dist_alertas = spark.sql("""
    SELECT 
        nivel_alerta_novo,
        prioridade_num,
        COUNT(*) as qtde,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentual,
        ROUND(SUM(saldo_credor_atual), 2) as saldo_total
    FROM vw_alertas_recalibrado
    GROUP BY nivel_alerta_novo, prioridade_num
    ORDER BY prioridade_num
""").toPandas()

print(dist_alertas.to_string(index=False))

# =============================================================================
# 6. TOP EMPRESAS POR NÍVEL DE ALERTA
# =============================================================================

print("\n" + "=" * 80)
print("📊 ETAPA 5: TOP EMPRESAS POR NÍVEL DE ALERTA")
print("=" * 80)

# 6.1 Ação Imediata
print("\n🔴 AÇÃO IMEDIATA - Top 20:")
print("-" * 60)

top_acao_imediata = spark.sql("""
    SELECT 
        ranking_risco,
        cnpj,
        nome_contribuinte,
        municipio,
        ROUND(score_total, 2) as score_total,
        qtde_indicios_graves,
        ROUND(saldo_credor_atual, 2) as saldo_credor
    FROM vw_alertas_recalibrado
    WHERE nivel_alerta_novo = 'AÇÃO IMEDIATA'
    ORDER BY score_total DESC
    LIMIT 20
""").toPandas()

if len(top_acao_imediata) > 0:
    print(top_acao_imediata.to_string(index=False))
else:
    print("   Nenhuma empresa neste nível")

# 6.2 Muito Urgente
print("\n🟠 MUITO URGENTE - Top 20:")
print("-" * 60)

top_muito_urgente = spark.sql("""
    SELECT 
        ranking_risco,
        cnpj,
        nome_contribuinte,
        municipio,
        ROUND(score_total, 2) as score_total,
        qtde_indicios_graves,
        ROUND(saldo_credor_atual, 2) as saldo_credor
    FROM vw_alertas_recalibrado
    WHERE nivel_alerta_novo = 'MUITO URGENTE'
    ORDER BY score_total DESC
    LIMIT 20
""").toPandas()

if len(top_muito_urgente) > 0:
    print(top_muito_urgente.to_string(index=False))
else:
    print("   Nenhuma empresa neste nível")

# 6.3 Urgente
print("\n🟡 URGENTE - Top 20:")
print("-" * 60)

top_urgente = spark.sql("""
    SELECT 
        ranking_risco,
        cnpj,
        nome_contribuinte,
        municipio,
        ROUND(score_total, 2) as score_total,
        total_protocolos,
        ROUND(saldo_credor_atual, 2) as saldo_credor
    FROM vw_alertas_recalibrado
    WHERE nivel_alerta_novo = 'URGENTE'
    ORDER BY score_total DESC
    LIMIT 20
""").toPandas()

if len(top_urgente) > 0:
    print(top_urgente.to_string(index=False))
else:
    print("   Nenhuma empresa neste nível")

# =============================================================================
# 7. RESUMO EXECUTIVO COM NOVOS LIMIARES
# =============================================================================

print("\n" + "=" * 80)
print("📋 RESUMO EXECUTIVO - RECALIBRAÇÃO")
print("=" * 80)

resumo_final = spark.sql(f"""
    SELECT 
        -- Totais
        COUNT(*) as total_empresas,
        
        -- Nova classificação de risco
        SUM(CASE WHEN classificacao_nova = 'CRÍTICO' THEN 1 ELSE 0 END) as risco_critico,
        SUM(CASE WHEN classificacao_nova = 'ALTO' THEN 1 ELSE 0 END) as risco_alto,
        SUM(CASE WHEN classificacao_nova = 'MÉDIO' THEN 1 ELSE 0 END) as risco_medio,
        SUM(CASE WHEN classificacao_nova = 'BAIXO' THEN 1 ELSE 0 END) as risco_baixo,
        
        -- Novos alertas
        SUM(CASE WHEN nivel_alerta_novo = 'AÇÃO IMEDIATA' THEN 1 ELSE 0 END) as acao_imediata,
        SUM(CASE WHEN nivel_alerta_novo = 'MUITO URGENTE' THEN 1 ELSE 0 END) as muito_urgente,
        SUM(CASE WHEN nivel_alerta_novo = 'URGENTE' THEN 1 ELSE 0 END) as urgente,
        SUM(CASE WHEN nivel_alerta_novo = 'PRIORIDADE ALTA' THEN 1 ELSE 0 END) as prioridade_alta,
        SUM(CASE WHEN nivel_alerta_novo = 'MONITORAR' THEN 1 ELSE 0 END) as monitorar,
        
        -- Scores médios
        ROUND(AVG(score_total), 2) as score_medio,
        ROUND(MAX(score_total), 2) as score_maximo,
        
        -- Saldos
        ROUND(SUM(saldo_credor_atual), 2) as saldo_total,
        ROUND(SUM(CASE WHEN nivel_alerta_novo IN ('AÇÃO IMEDIATA', 'MUITO URGENTE') 
                       THEN saldo_credor_atual ELSE 0 END), 2) as saldo_urgente
        
    FROM vw_alertas_recalibrado
""").collect()[0]

print(f"""
📊 INDICADORES COM NOVOS LIMIARES:
{'='*60}

🎯 CLASSIFICAÇÃO DE RISCO:
   • CRÍTICO (score >= {p95}):  {resumo_final['risco_critico']:,} empresas
   • ALTO (score >= {p90}):     {resumo_final['risco_alto']:,} empresas
   • MÉDIO (score >= {p75}):    {resumo_final['risco_medio']:,} empresas
   • BAIXO (score < {p75}):     {resumo_final['risco_baixo']:,} empresas

🚨 NÍVEIS DE ALERTA:
   🔴 AÇÃO IMEDIATA:    {resumo_final['acao_imediata']:,} empresas
   🟠 MUITO URGENTE:    {resumo_final['muito_urgente']:,} empresas
   🟡 URGENTE:          {resumo_final['urgente']:,} empresas
   ⚪ PRIORIDADE ALTA:  {resumo_final['prioridade_alta']:,} empresas
   ⚫ MONITORAR:        {resumo_final['monitorar']:,} empresas

📈 MÉTRICAS:
   • Score Médio:       {resumo_final['score_medio']}
   • Score Máximo:      {resumo_final['score_maximo']}
   • Saldo Total:       R$ {resumo_final['saldo_total']:,.2f}
   • Saldo Urgente:     R$ {resumo_final['saldo_urgente']:,.2f}
""")

# =============================================================================
# 8. SALVAR TABELA COM NOVOS LIMIARES
# =============================================================================

print("\n" + "=" * 80)
print("📊 ETAPA 6: SALVANDO TABELA RECALIBRADA")
print("=" * 80)

# Criar tabela definitiva com novos limiares
spark.sql(f"""
    DROP TABLE IF EXISTS teste.luciano_scores_v2
""")

spark.sql(f"""
    CREATE TABLE teste.luciano_scores_v2 AS
    SELECT 
        cnpj,
        nome_contribuinte,
        municipio,
        score_comportamento,
        score_credito,
        score_indicios,
        score_total,
        total_protocolos,
        saldo_credor_atual,
        qtde_indicios,
        qtde_indicios_graves,
        flag_cancelada,
        flag_reincidente,
        taxa_permanencia,
        
        -- Classificações antigas (para comparação)
        classificacao_risco_final AS classificacao_antiga,
        
        -- Novas classificações
        classificacao_nova AS classificacao_risco,
        nivel_alerta_novo AS nivel_alerta,
        prioridade_num AS prioridade,
        ranking_risco,
        
        -- Limiares utilizados (para referência)
        {p95} AS limiar_critico,
        {p90} AS limiar_alto,
        {p75} AS limiar_medio
        
    FROM vw_alertas_recalibrado
""")

total_salvo = spark.sql("SELECT COUNT(*) as total FROM teste.luciano_scores_v2").collect()[0]['total']
print(f"✅ Tabela teste.luciano_scores_v2 criada com {total_salvo:,} registros")

# =============================================================================
# 9. EXPORTAR LIMIARES SUGERIDOS PARA O DASHBOARD
# =============================================================================

print("\n" + "=" * 80)
print("📋 LIMIARES SUGERIDOS PARA O DASHBOARD")
print("=" * 80)

print(f"""
# Copie estes valores para o arquivo luciano_dashboard.py:

# LIMIARES DE CLASSIFICAÇÃO DE RISCO (baseados em percentis)
LIMIAR_CRITICO = {p95}  # Percentil 95
LIMIAR_ALTO = {p90}     # Percentil 90
LIMIAR_MEDIO = {p75}    # Percentil 75

# Função de classificação atualizada:
def classificar_risco(score):
    if score >= {p95}:
        return 'CRÍTICO'
    elif score >= {p90}:
        return 'ALTO'
    elif score >= {p75}:
        return 'MÉDIO'
    else:
        return 'BAIXO'

# Função de alerta atualizada:
def classificar_alerta(row):
    score = row.get('score_total', 0)
    indicios_graves = row.get('qtde_indicios_graves', 0)
    saldo = row.get('saldo_credor_atual', 0)
    reincidente = row.get('flag_empresa_reincidente', 0)
    protocolos = row.get('total_protocolos', 0)
    
    # Classificação de risco
    if score >= {p95}:
        classif = 'CRÍTICO'
    elif score >= {p90}:
        classif = 'ALTO'
    elif score >= {p75}:
        classif = 'MÉDIO'
    else:
        classif = 'BAIXO'
    
    # Nível de alerta
    if classif == 'CRÍTICO' and (indicios_graves >= 1 or saldo > 50000):
        return 'AÇÃO IMEDIATA', 1
    elif classif == 'CRÍTICO':
        return 'MUITO URGENTE', 2
    elif classif == 'ALTO' and (indicios_graves >= 1 or reincidente == 1):
        return 'MUITO URGENTE', 2
    elif classif == 'ALTO':
        return 'URGENTE', 3
    elif classif == 'MÉDIO' and (protocolos >= 3 or saldo > 10000):
        return 'URGENTE', 3
    elif classif == 'MÉDIO':
        return 'PRIORIDADE ALTA', 4
    else:
        return 'MONITORAR', 5
""")

print("\n" + "=" * 80)
print("✅ RECALIBRAÇÃO CONCLUÍDA!")
print("=" * 80)

PROJETO LUCIANO - RECALIBRAÇÃO DE LIMIARES

📊 ETAPA 1: ANALISANDO DISTRIBUIÇÃO DOS SCORES
------------------------------------------------------------


✅ Total de empresas na base: 243,053

📈 ESTATÍSTICAS DO SCORE_TOTAL:
----------------------------------------


   Média:           9.08
   Desvio Padrão:   5.63
   Mínimo:          1.25
   Máximo:          48.5
   Percentil 25%:   6.25
   Mediana (50%):   6.25
   Percentil 75%:   9.75
   Percentil 90%:   15.0
   Percentil 95%:   21.0
   Percentil 99%:   30.0

📊 ESTATÍSTICAS POR COMPONENTE:
----------------------------------------


         componente  media  maximo  p95
score_comportamento  27.79    90.0 55.0
      score_credito   3.26    70.0 25.0
     score_indicios   2.49    50.0 15.0

📊 ETAPA 2: DEFININDO NOVOS LIMIARES

🎯 LIMIARES ANTIGOS vs NOVOS:
------------------------------------------------------------
Classificação   Limiar Antigo        Limiar Novo (Sugerido)   
------------------------------------------------------------
CRÍTICO         score >= 70          score >= 21.0             (Top 5%)
ALTO            score >= 50          score >= 15.0             (Top 10%)
MÉDIO           score >= 30          score >= 9.75             (Top 25%)
BAIXO           score < 30           score < 9.75              (Restante)

📊 ETAPA 3: APLICANDO NOVOS LIMIARES

📊 DISTRIBUIÇÃO COM NOVOS LIMIARES:
------------------------------------------------------------
classificacao_nova   qtde percentual  saldo_total  score_medio
           CRÍTICO  15896       6.54     3.51e+08        25.55
              ALTO  27668      11.38

✅ Tabela teste.luciano_scores_v2 criada com 243,053 registros

📋 LIMIARES SUGERIDOS PARA O DASHBOARD

# Copie estes valores para o arquivo luciano_dashboard.py:

# LIMIARES DE CLASSIFICAÇÃO DE RISCO (baseados em percentis)
LIMIAR_CRITICO = 21.0  # Percentil 95
LIMIAR_ALTO = 15.0     # Percentil 90
LIMIAR_MEDIO = 9.75    # Percentil 75

# Função de classificação atualizada:
def classificar_risco(score):
    if score >= 21.0:
        return 'CRÍTICO'
    elif score >= 15.0:
        return 'ALTO'
    elif score >= 9.75:
        return 'MÉDIO'
    else:
        return 'BAIXO'

# Função de alerta atualizada:
def classificar_alerta(row):
    score = row.get('score_total', 0)
    indicios_graves = row.get('qtde_indicios_graves', 0)
    saldo = row.get('saldo_credor_atual', 0)
    reincidente = row.get('flag_empresa_reincidente', 0)
    protocolos = row.get('total_protocolos', 0)
    
    # Classificação de risco
    if score >= 21.0:
        classif = 'CRÍTICO'
    elif score >= 15.0:
       

In [6]:
print("🔄 Atualizando metadados da tabela base...")
spark.sql("REFRESH TABLE teste.cancel_luciano")

🔄 Atualizando metadados da tabela base...


DataFrame[]

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
ANÁLISE EXPLORATÓRIA - CANCELAMENTOS FISCAL LUCIANO TREVISAN
================================================================================
Este script lê a tabela teste.cancel_luciano (criada via Impala) e gera
análises exploratórias e resumo executivo para relatório.

PREREQUISITO: Executar cancel_luciano_impala.sql no Impala antes deste script
================================================================================
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURAÇÃO
# ==============================================================================

spark = SparkSession.builder \
    .appName("Analise_Cancel_Luciano") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "50") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# ==============================================================================
# FUNÇÕES AUXILIARES
# ==============================================================================

def formatar_br(valor):
    """Formata valor numérico para padrão brasileiro (1.234.567,89)"""
    if valor is None:
        return "0,00"
    try:
        valor = float(valor)
        if valor >= 1e9:
            return f"{valor/1e9:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") + " bi"
        elif valor >= 1e6:
            return f"{valor/1e6:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") + " mi"
        else:
            return f"{valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    except:
        return str(valor)

def formatar_perc(valor):
    """Formata percentual"""
    if valor is None:
        return "0,0%"
    return f"{valor:.1f}%".replace(".", ",")

def linha(char="=", tamanho=100):
    print(char * tamanho)

def titulo(texto):
    linha("=")
    print(f"📊 {texto}")
    linha("=")

def subtitulo(texto):
    print(f"\n{'─'*80}")
    print(f"  {texto}")
    print(f"{'─'*80}")

# ==============================================================================
# CARREGAR DADOS
# ==============================================================================

titulo("CARREGANDO DADOS DA TABELA teste.cancel_luciano")

try:
    df = spark.sql("SELECT * FROM teste.cancel_luciano")
    total_registros = df.count()
    print(f"\n✅ Tabela carregada com sucesso!")
    print(f"   Total de registros: {total_registros:,}".replace(",", "."))
except Exception as e:
    print(f"\n❌ ERRO ao carregar tabela: {e}")
    print("\n⚠️  Execute primeiro o script cancel_luciano_impala.sql no Impala!")
    spark.stop()
    exit(1)

# Cache para performance
df.cache()

# ==============================================================================
# 1. VISÃO GERAL
# ==============================================================================

titulo("1. VISÃO GERAL DOS CANCELAMENTOS")

# Métricas básicas
total_empresas = df.select("cnpj").distinct().count()
total_protocolos = df.count()

stats = df.agg(
    F.sum(F.when(F.col("flag_ainda_cancelada") == 1, 1).otherwise(0)).alias("ainda_canceladas"),
    F.sum(F.when(F.col("flag_reativada") == 1, 1).otherwise(0)).alias("reativadas"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.sum("qtde_indicios").alias("total_indicios"),
    F.sum("qtde_indicios_graves").alias("total_indicios_graves"),
    F.sum("qtde_indicios_criticos").alias("total_indicios_criticos"),
    F.sum("qtde_nfe_emitidas").alias("total_nfe_emitidas"),
    F.sum("vl_total_nfe_emitidas").alias("vl_total_nfe_emitidas"),
    F.sum("qtde_nfe_recebidas").alias("total_nfe_recebidas"),
    F.sum("vl_total_nfe_recebidas").alias("vl_total_nfe_recebidas"),
    F.sum("valor_pagamentos").alias("total_recolhido"),
    F.avg("score_prioridade_acao").alias("score_medio")
).collect()[0]

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                        RESUMO EXECUTIVO                                      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Total de Protocolos de Cancelamento:  {total_protocolos:>8,}                           │
│  Empresas Distintas:                   {total_empresas:>8,}                           │
├─────────────────────────────────────────────────────────────────────────────┤
│  Ainda Canceladas:                     {stats['ainda_canceladas']:>8,}  ({100*stats['ainda_canceladas']/total_protocolos:.1f}%)              │
│  Reativadas:                           {stats['reativadas']:>8,}  ({100*stats['reativadas']/total_protocolos:.1f}%)              │
├─────────────────────────────────────────────────────────────────────────────┤
│  Saldo Credor Total:             R$ {formatar_br(stats['saldo_total']):>15}                       │
│  Total Recolhido (pagamentos):   R$ {formatar_br(stats['total_recolhido']):>15}                       │
├─────────────────────────────────────────────────────────────────────────────┤
│  Total de Indícios:                    {stats['total_indicios']:>8,}                           │
│  Indícios Graves (score>=50):          {stats['total_indicios_graves']:>8,}                           │
│  Indícios Críticos (score>=80):        {stats['total_indicios_criticos']:>8,}                           │
├─────────────────────────────────────────────────────────────────────────────┤
│  NF-e Emitidas:                        {stats['total_nfe_emitidas']:>8,}                           │
│  Valor NF-e Emitidas:            R$ {formatar_br(stats['vl_total_nfe_emitidas']):>15}                       │
│  NF-e Recebidas:                       {stats['total_nfe_recebidas']:>8,}                           │
│  Valor NF-e Recebidas:           R$ {formatar_br(stats['vl_total_nfe_recebidas']):>15}                       │
├─────────────────────────────────────────────────────────────────────────────┤
│  Score Médio de Prioridade:            {stats['score_medio']:>8.1f}                           │
└─────────────────────────────────────────────────────────────────────────────┘
""".replace(",", "."))


# ==============================================================================
# 2. DISTRIBUIÇÃO POR CLASSIFICAÇÃO DE RISCO
# ==============================================================================

titulo("2. DISTRIBUIÇÃO POR CLASSIFICAÇÃO DE RISCO")

risco_df = df.groupBy("classificacao_risco_final").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.sum("qtde_indicios_graves").alias("indicios_graves"),
    F.sum("qtde_nfe_emitidas").alias("nfe_emitidas"),
    F.sum("vl_total_nfe_emitidas").alias("vl_nfe")
).orderBy(
    F.when(F.col("classificacao_risco_final") == "CRÍTICO", 1)
    .when(F.col("classificacao_risco_final") == "ALTO", 2)
    .when(F.col("classificacao_risco_final") == "MÉDIO", 3)
    .otherwise(4)
).collect()

print(f"\n{'Risco':<12} {'Qtde':>8} {'%':>8} {'Saldo Credor':>18} {'Ind.Graves':>12} {'NF-e Emit':>12} {'Valor NF-e':>18}")
print("-" * 100)

for row in risco_df:
    perc = 100 * row['qtde'] / total_protocolos
    print(f"{row['classificacao_risco_final']:<12} {row['qtde']:>8,} {perc:>7.1f}% R$ {formatar_br(row['saldo_total']):>14} {row['indicios_graves']:>12,} {row['nfe_emitidas']:>12,} R$ {formatar_br(row['vl_nfe']):>14}".replace(",", "."))

# Resumo por risco
print(f"\n📌 ALERTAS POR RISCO:")
for row in risco_df:
    if row['classificacao_risco_final'] in ['CRÍTICO', 'ALTO']:
        print(f"   ⚠️  {row['classificacao_risco_final']}: {row['qtde']} empresas com R$ {formatar_br(row['saldo_total'])} em saldo credor")


# ==============================================================================
# 3. DISTRIBUIÇÃO POR CATEGORIA DE COMPORTAMENTO
# ==============================================================================

titulo("3. DISTRIBUIÇÃO POR CATEGORIA DE COMPORTAMENTO")

cat_df = df.groupBy("categoria_comportamento").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.avg("score_prioridade_acao").alias("score_medio"),
    F.sum("qtde_indicios").alias("total_indicios"),
    F.sum("qtde_nfe_emitidas").alias("nfe_emitidas")
).orderBy(F.desc("qtde")).collect()

print(f"\n{'Categoria':<25} {'Qtde':>8} {'%':>8} {'Saldo Credor':>18} {'Score Médio':>12} {'Indícios':>10}")
print("-" * 100)

for row in cat_df:
    perc = 100 * row['qtde'] / total_protocolos
    print(f"{row['categoria_comportamento']:<25} {row['qtde']:>8,} {perc:>7.1f}% R$ {formatar_br(row['saldo_total']):>14} {row['score_medio']:>12.1f} {row['total_indicios']:>10,}".replace(",", "."))


# ==============================================================================
# 4. DISTRIBUIÇÃO POR MOTIVO DE CANCELAMENTO
# ==============================================================================

titulo("4. DISTRIBUIÇÃO POR MOTIVO DE CANCELAMENTO")

motivo_df = df.groupBy("cod_motivo", "descricao_motivo_cancelamento").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.sum(F.when(F.col("flag_ainda_cancelada") == 1, 1).otherwise(0)).alias("ainda_cancel")
).orderBy(F.desc("qtde")).collect()

print(f"\n{'Cód':>4} {'Motivo':<35} {'Qtde':>8} {'%':>8} {'Canceladas':>12} {'Saldo Credor':>18}")
print("-" * 100)

for row in motivo_df:
    perc = 100 * row['qtde'] / total_protocolos
    motivo = row['descricao_motivo_cancelamento'][:35] if row['descricao_motivo_cancelamento'] else "N/A"
    cod = row['cod_motivo'] if row['cod_motivo'] is not None else 0
    print(f"{cod:>4} {motivo:<35} {row['qtde']:>8,} {perc:>7.1f}% {row['ainda_cancel']:>12,} R$ {formatar_br(row['saldo_total']):>14}".replace(",", "."))


# ==============================================================================
# 5. ANÁLISE DE OMISSÕES
# ==============================================================================

titulo("5. ANÁLISE DE OMISSÕES (DIME E PGDAS)")

omissoes = df.agg(
    # DIME
    F.sum(F.when(F.col("periodos_omissos_dime") > 0, 1).otherwise(0)).alias("emp_omissao_dime"),
    F.sum("periodos_omissos_dime").alias("total_periodos_omissos_dime"),
    F.sum("periodos_zerados_dime").alias("total_periodos_zerados_dime"),
    F.sum(F.when(F.col("flag_omissao_dime") == 1, 1).otherwise(0)).alias("flag_omissao_dime"),
    # PGDAS
    F.sum(F.when(F.col("periodos_omissos_pgdas") > 0, 1).otherwise(0)).alias("emp_omissao_pgdas"),
    F.sum("periodos_omissos_pgdas").alias("total_periodos_omissos_pgdas"),
    F.sum("periodos_zerados_pgdas").alias("total_periodos_zerados_pgdas"),
    F.sum(F.when(F.col("flag_omissao_pgdas") == 1, 1).otherwise(0)).alias("flag_omissao_pgdas"),
    # Valores
    F.sum("vl_faturamento_dime").alias("total_faturamento_dime"),
    F.sum("vl_receita_bruta_pgdas").alias("total_receita_pgdas")
).collect()[0]

print(f"""
┌───────────────────────────────────────────────────────────────────────────┐
│                         REGIME NORMAL (DIME)                               │
├───────────────────────────────────────────────────────────────────────────┤
│  Empresas com períodos omissos:        {omissoes['emp_omissao_dime']:>8,}                         │
│  Total de períodos omissos:            {omissoes['total_periodos_omissos_dime']:>8,}                         │
│  Total de períodos zerados:            {omissoes['total_periodos_zerados_dime']:>8,}                         │
│  Com indício de omissão DIME:          {omissoes['flag_omissao_dime']:>8,}                         │
│  Faturamento declarado total:    R$ {formatar_br(omissoes['total_faturamento_dime']):>15}                   │
├───────────────────────────────────────────────────────────────────────────┤
│                       SIMPLES NACIONAL (PGDAS)                             │
├───────────────────────────────────────────────────────────────────────────┤
│  Empresas com períodos omissos:        {omissoes['emp_omissao_pgdas']:>8,}                         │
│  Total de períodos omissos:            {omissoes['total_periodos_omissos_pgdas']:>8,}                         │
│  Total de períodos zerados:            {omissoes['total_periodos_zerados_pgdas']:>8,}                         │
│  Com indício de omissão PGDAS:         {omissoes['flag_omissao_pgdas']:>8,}                         │
│  Receita bruta declarada total:  R$ {formatar_br(omissoes['total_receita_pgdas']):>15}                   │
└───────────────────────────────────────────────────────────────────────────┘
""".replace(",", "."))


# ==============================================================================
# 6. ANÁLISE DE NF-e vs DECLARAÇÕES
# ==============================================================================

titulo("6. ANÁLISE DE NF-e vs DECLARAÇÕES")

nfe_decl = df.agg(
    # Empresas com NF-e
    F.sum(F.when(F.col("qtde_nfe_emitidas") > 0, 1).otherwise(0)).alias("emp_com_nfe"),
    # NF-e sem declaração (DIME ou PGDAS)
    F.sum(F.when(
        (F.col("qtde_nfe_emitidas") > 0) & 
        (F.col("vl_faturamento_dime") == 0) & 
        (F.col("vl_receita_bruta_pgdas") == 0), 1
    ).otherwise(0)).alias("nfe_sem_declaracao"),
    # Valor das NF-e sem declaração
    F.sum(F.when(
        (F.col("qtde_nfe_emitidas") > 0) & 
        (F.col("vl_faturamento_dime") == 0) & 
        (F.col("vl_receita_bruta_pgdas") == 0), 
        F.col("vl_total_nfe_emitidas")
    ).otherwise(0)).alias("vl_nfe_sem_declaracao"),
    # Empresas com declaração mas sem NF-e
    F.sum(F.when(
        (F.col("qtde_nfe_emitidas") == 0) & 
        ((F.col("vl_faturamento_dime") > 0) | (F.col("vl_receita_bruta_pgdas") > 0)), 1
    ).otherwise(0)).alias("declaracao_sem_nfe")
).collect()[0]

print(f"""
┌───────────────────────────────────────────────────────────────────────────┐
│                    CRUZAMENTO NF-e x DECLARAÇÕES                          │
├───────────────────────────────────────────────────────────────────────────┤
│  Empresas que emitiram NF-e:           {nfe_decl['emp_com_nfe']:>8,}                         │
│                                                                           │
│  ⚠️  NF-e SEM declaração (DIME/PGDAS): {nfe_decl['nfe_sem_declaracao']:>8,}                         │
│      Valor dessas NF-e:          R$ {formatar_br(nfe_decl['vl_nfe_sem_declaracao']):>15}                   │
│                                                                           │
│  Declaração SEM NF-e emitida:          {nfe_decl['declaracao_sem_nfe']:>8,}                         │
└───────────────────────────────────────────────────────────────────────────┘
""".replace(",", "."))

if nfe_decl['nfe_sem_declaracao'] > 0:
    print(f"\n🚨 ALERTA: {nfe_decl['nfe_sem_declaracao']} empresas emitiram NF-e mas não declararam faturamento!")
    print(f"   Valor potencial não declarado: R$ {formatar_br(nfe_decl['vl_nfe_sem_declaracao'])}")


# ==============================================================================
# 7. DISTRIBUIÇÃO POR MUNICÍPIO
# ==============================================================================

titulo("7. TOP 15 MUNICÍPIOS COM MAIS CANCELAMENTOS")

mun_df = df.groupBy("municipio").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.sum(F.when(F.col("flag_ainda_cancelada") == 1, 1).otherwise(0)).alias("ainda_cancel"),
    F.sum("qtde_indicios_graves").alias("indicios_graves")
).orderBy(F.desc("qtde")).limit(15).collect()

print(f"\n{'Município':<30} {'Qtde':>8} {'%':>8} {'Canceladas':>12} {'Saldo Credor':>18} {'Ind.Graves':>12}")
print("-" * 100)

for row in mun_df:
    perc = 100 * row['qtde'] / total_protocolos
    mun = row['municipio'][:30] if row['municipio'] else "N/A"
    print(f"{mun:<30} {row['qtde']:>8,} {perc:>7.1f}% {row['ainda_cancel']:>12,} R$ {formatar_br(row['saldo_total']):>14} {row['indicios_graves']:>12,}".replace(",", "."))


# ==============================================================================
# 8. DISTRIBUIÇÃO POR GERÊNCIA REGIONAL
# ==============================================================================

titulo("8. DISTRIBUIÇÃO POR GERÊNCIA REGIONAL")

gr_df = df.groupBy("gerencia_regional").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total"),
    F.sum(F.when(F.col("classificacao_risco_final").isin(["CRÍTICO", "ALTO"]), 1).otherwise(0)).alias("alto_risco"),
    F.avg("score_prioridade_acao").alias("score_medio")
).orderBy(F.desc("qtde")).collect()

print(f"\n{'Gerência Regional':<25} {'Qtde':>8} {'%':>8} {'Alto Risco':>12} {'Saldo Credor':>18} {'Score Médio':>12}")
print("-" * 100)

for row in gr_df:
    perc = 100 * row['qtde'] / total_protocolos
    gr = row['gerencia_regional'][:25] if row['gerencia_regional'] else "N/A"
    print(f"{gr:<25} {row['qtde']:>8,} {perc:>7.1f}% {row['alto_risco']:>12,} R$ {formatar_br(row['saldo_total']):>14} {row['score_medio']:>12.1f}".replace(",", "."))


# ==============================================================================
# 9. DISTRIBUIÇÃO TEMPORAL
# ==============================================================================

titulo("9. DISTRIBUIÇÃO TEMPORAL DOS CANCELAMENTOS")

# Por ano
temporal_df = df.withColumn("ano_cancelamento", F.year("data_cancelamento"))

ano_df = temporal_df.groupBy("ano_cancelamento").agg(
    F.count("*").alias("qtde"),
    F.sum("saldo_credor_atual").alias("saldo_total")
).orderBy("ano_cancelamento").collect()

print(f"\n{'Ano':>6} {'Qtde':>10} {'%':>10} {'Saldo Credor':>20}")
print("-" * 60)

for row in ano_df:
    if row['ano_cancelamento']:
        perc = 100 * row['qtde'] / total_protocolos
        print(f"{row['ano_cancelamento']:>6} {row['qtde']:>10,} {perc:>9.1f}% R$ {formatar_br(row['saldo_total']):>16}".replace(",", "."))


# ==============================================================================
# 10. TOP 20 EMPRESAS PRIORITÁRIAS
# ==============================================================================

titulo("10. TOP 20 EMPRESAS PRIORITÁRIAS (MAIOR SCORE)")

top20 = df.orderBy(F.desc("score_prioridade_acao")).limit(20).collect()

print(f"\n{'#':>3} {'CNPJ':<18} {'Empresa':<30} {'Município':<15} {'Risco':<10} {'Score':>6} {'Saldo Credor':>16} {'Ind.Graves':>10}")
print("-" * 130)

for i, row in enumerate(top20, 1):
    nome = row['nome_contribuinte'][:30] if row['nome_contribuinte'] else "N/A"
    mun = row['municipio'][:15] if row['municipio'] else "N/A"
    risco = row['classificacao_risco_final'] or "N/A"
    print(f"{i:>3} {row['cnpj']:<18} {nome:<30} {mun:<15} {risco:<10} {row['score_prioridade_acao']:>6} R$ {formatar_br(row['saldo_credor_atual']):>12} {row['qtde_indicios_graves']:>10}".replace(",", "."))

# Detalhamento das top 5
subtitulo("DETALHAMENTO DAS TOP 5 EMPRESAS")

for i, row in enumerate(top20[:5], 1):
    print(f"""
{'─'*80}
📍 #{i} - {row['nome_contribuinte']}
{'─'*80}
   CNPJ: {row['cnpj']}  |  IE: {row['ie']}
   Município: {row['municipio']} ({row['gerencia_regional']})
   CNAE: {row['cd_cnae']} - {row['descricao_cnae'][:50] if row['descricao_cnae'] else 'N/A'}
   
   📅 Cancelamento: {row['data_cancelamento']}
   📝 Motivo: {row['descricao_motivo_cancelamento']}
   🔄 Situação: {'AINDA CANCELADA' if row['flag_ainda_cancelada'] == 1 else 'REATIVADA'}
   
   💰 Saldo Credor: R$ {formatar_br(row['saldo_credor_atual'])}
   📊 Score: {row['score_prioridade_acao']} pontos | Risco: {row['classificacao_risco_final']}
   🏷️  Categoria: {row['categoria_comportamento']}
   
   ⚠️  Indícios: {row['qtde_indicios']} total | {row['qtde_indicios_graves']} graves | {row['qtde_indicios_criticos']} críticos
   📄 NF-e Emitidas: {row['qtde_nfe_emitidas']:,} (R$ {formatar_br(row['vl_total_nfe_emitidas'])})
   📥 NF-e Recebidas: {row['qtde_nfe_recebidas']:,} (R$ {formatar_br(row['vl_total_nfe_recebidas'])})
   
   📋 Omissões DIME: {row['periodos_omissos_dime']} períodos | Zerados: {row['periodos_zerados_dime']}
   📋 Omissões PGDAS: {row['periodos_omissos_pgdas']} períodos | Zerados: {row['periodos_zerados_pgdas']}
""".replace(",", "."))


# ==============================================================================
# 11. EMPRESAS COM SALDO ALTO AINDA CANCELADAS
# ==============================================================================

titulo("11. EMPRESAS COM SALDO ALTO AINDA CANCELADAS (> R$ 100.000)")

saldo_alto = df.filter(
    (F.col("flag_ainda_cancelada") == 1) & 
    (F.col("saldo_credor_atual") > 100000)
).orderBy(F.desc("saldo_credor_atual")).limit(15).collect()

if saldo_alto:
    print(f"\n{'CNPJ':<18} {'Empresa':<35} {'Saldo Credor':>18} {'Risco':<10}")
    print("-" * 90)
    
    total_saldo_alto = 0
    for row in saldo_alto:
        nome = row['nome_contribuinte'][:35] if row['nome_contribuinte'] else "N/A"
        print(f"{row['cnpj']:<18} {nome:<35} R$ {formatar_br(row['saldo_credor_atual']):>14} {row['classificacao_risco_final']:<10}")
        total_saldo_alto += row['saldo_credor_atual'] or 0
    
    print("-" * 90)
    print(f"{'TOTAL':<54} R$ {formatar_br(total_saldo_alto):>14}")
    print(f"\n🚨 ALERTA: {len(saldo_alto)} empresas canceladas com saldo credor > R$ 100.000!")
else:
    print("\n✅ Nenhuma empresa cancelada com saldo credor > R$ 100.000")


# ==============================================================================
# 12. RESUMO FINAL PARA RELATÓRIO
# ==============================================================================

titulo("12. RESUMO FINAL PARA RELATÓRIO")

# Contagens por categoria
critico = df.filter(F.col("classificacao_risco_final") == "CRÍTICO").count()
alto = df.filter(F.col("classificacao_risco_final") == "ALTO").count()
medio = df.filter(F.col("classificacao_risco_final") == "MÉDIO").count()
baixo = df.filter(F.col("classificacao_risco_final") == "BAIXO").count()

# Valores
saldo_critico = df.filter(F.col("classificacao_risco_final") == "CRÍTICO").agg(F.sum("saldo_credor_atual")).collect()[0][0] or 0
saldo_alto = df.filter(F.col("classificacao_risco_final") == "ALTO").agg(F.sum("saldo_credor_atual")).collect()[0][0] or 0

print(f"""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   RELATÓRIO DE CANCELAMENTOS - FISCAL LUCIANO TREVISAN FREITAS               ║
║   Data de Geração: {datetime.now().strftime('%d/%m/%Y %H:%M')}                                          ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║   VOLUME DE TRABALHO                                                          ║
║   ─────────────────                                                           ║
║   • Total de protocolos de cancelamento: {total_protocolos:>6,}                            ║
║   • Empresas distintas processadas:      {total_empresas:>6,}                            ║
║   • Empresas ainda canceladas:           {stats['ainda_canceladas']:>6,} ({100*stats['ainda_canceladas']/total_protocolos:>5.1f}%)           ║
║   • Empresas reativadas:                 {stats['reativadas']:>6,} ({100*stats['reativadas']/total_protocolos:>5.1f}%)           ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║   CLASSIFICAÇÃO DE RISCO                                                      ║
║   ──────────────────────                                                      ║
║   🔴 CRÍTICO:  {critico:>5} empresas  │  Saldo: R$ {formatar_br(saldo_critico):>15}              ║
║   🟠 ALTO:     {alto:>5} empresas  │  Saldo: R$ {formatar_br(saldo_alto):>15}              ║
║   🟡 MÉDIO:    {medio:>5} empresas                                               ║
║   🟢 BAIXO:    {baixo:>5} empresas                                               ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║   INDICADORES DE IRREGULARIDADE                                               ║
║   ─────────────────────────────                                               ║
║   • Total de indícios identificados:     {stats['total_indicios']:>6,}                            ║
║   • Indícios graves (score >= 50):       {stats['total_indicios_graves']:>6,}                            ║
║   • Indícios críticos (score >= 80):     {stats['total_indicios_criticos']:>6,}                            ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║   MOVIMENTAÇÃO FISCAL                                                         ║
║   ───────────────────                                                         ║
║   • NF-e Emitidas:     {stats['total_nfe_emitidas']:>10,}  │  Valor: R$ {formatar_br(stats['vl_total_nfe_emitidas']):>15}   ║
║   • NF-e Recebidas:    {stats['total_nfe_recebidas']:>10,}  │  Valor: R$ {formatar_br(stats['vl_total_nfe_recebidas']):>15}   ║
║   • Recolhimento:                      R$ {formatar_br(stats['total_recolhido']):>15}   ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║   VALORES EM RISCO                                                            ║
║   ────────────────                                                            ║
║   • Saldo credor total das empresas:  R$ {formatar_br(stats['saldo_total']):>15}              ║
║   • Saldo em empresas CRÍTICO+ALTO:   R$ {formatar_br(saldo_critico + saldo_alto):>15}              ║
║   • Empresas com NF-e sem declaração:    {nfe_decl['nfe_sem_declaracao']:>6,}                            ║
║   • Valor NF-e não declarado:         R$ {formatar_br(nfe_decl['vl_nfe_sem_declaracao']):>15}              ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
""".replace(",", "."))


# ==============================================================================
# EXPORTAR DADOS PARA ANÁLISE EXTERNA (OPCIONAL)
# ==============================================================================

subtitulo("EXPORTANDO RESUMOS PARA ANÁLISE")

try:
    # Dropar tabelas se existirem
    spark.sql("DROP TABLE IF EXISTS teste.cancel_luciano_resumo_risco")
    spark.sql("DROP TABLE IF EXISTS teste.cancel_luciano_resumo_categoria")
    spark.sql("DROP TABLE IF EXISTS teste.cancel_luciano_prioritarias")
    
    # Criar views temporárias para exportação
    df.createOrReplaceTempView("cancel_luciano")
    
    # Salvar resumo por risco
    spark.sql("""
        CREATE TABLE teste.cancel_luciano_resumo_risco AS
        SELECT 
            classificacao_risco_final,
            COUNT(*) as qtde,
            SUM(saldo_credor_atual) as saldo_total,
            SUM(qtde_indicios_graves) as indicios_graves,
            SUM(qtde_nfe_emitidas) as nfe_emitidas
        FROM cancel_luciano
        GROUP BY classificacao_risco_final
    """)
    print("✅ Tabela teste.cancel_luciano_resumo_risco criada")
    
    # Salvar resumo por categoria
    spark.sql("""
        CREATE TABLE teste.cancel_luciano_resumo_categoria AS
        SELECT 
            categoria_comportamento,
            COUNT(*) as qtde,
            SUM(saldo_credor_atual) as saldo_total,
            AVG(score_prioridade_acao) as score_medio
        FROM cancel_luciano
        GROUP BY categoria_comportamento
    """)
    print("✅ Tabela teste.cancel_luciano_resumo_categoria criada")
    
    # Salvar top prioritárias
    spark.sql("""
        CREATE TABLE teste.cancel_luciano_prioritarias AS
        SELECT *
        FROM cancel_luciano
        WHERE classificacao_risco_final IN ('CRÍTICO', 'ALTO')
    """)
    print("✅ Tabela teste.cancel_luciano_prioritarias criada")
    
except Exception as e:
    print(f"⚠️  Não foi possível criar tabelas de resumo: {e}")
    print("   (A análise já foi concluída, tabelas de resumo são opcionais)")

print(f"""
{'='*80}
✅ ANÁLISE CONCLUÍDA COM SUCESSO!
{'='*80}

Tabelas criadas:
• teste.cancel_luciano              - Dados completos
• teste.cancel_luciano_resumo_risco     - Resumo por classificação de risco
• teste.cancel_luciano_resumo_categoria - Resumo por categoria de comportamento
• teste.cancel_luciano_prioritarias     - Empresas de alto risco para ação

Data: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}
""")

# Limpar cache
df.unpersist()

spark.stop()

📊 CARREGANDO DADOS DA TABELA teste.cancel_luciano

✅ Tabela carregada com sucesso!
   Total de registros: 4.928
📊 1. VISÃO GERAL DOS CANCELAMENTOS

┌─────────────────────────────────────────────────────────────────────────────┐
│                        RESUMO EXECUTIVO                                      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Total de Protocolos de Cancelamento:     4.928                           │
│  Empresas Distintas:                      4.724                           │
├─────────────────────────────────────────────────────────────────────────────┤
│  Ainda Canceladas:                        3.577  (72.6%)              │
│  Reativadas:                              1.351  (27.4%)              │
├─────────────────────────────────────────────────────────────────────────────┤
│  Saldo Credor Total:             R$       105.57 mi                       │
│  Total Recolhido (pagamentos):   R$         5.85 mi                

✅ Tabela teste.cancel_luciano_resumo_risco criada
✅ Tabela teste.cancel_luciano_resumo_categoria criada


✅ Tabela teste.cancel_luciano_prioritarias criada

✅ ANÁLISE CONCLUÍDA COM SUCESSO!

Tabelas criadas:
• teste.cancel_luciano              - Dados completos
• teste.cancel_luciano_resumo_risco     - Resumo por classificação de risco
• teste.cancel_luciano_resumo_categoria - Resumo por categoria de comportamento
• teste.cancel_luciano_prioritarias     - Empresas de alto risco para ação

Data: 01/12/2025 16:42:18



Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:551)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:519)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
